In [ ]:
import openmc
import numpy as np

In [ ]:
# Create materials
fuel = openmc.Material(name='fuel')
fuel.set_density('g/cm3', density=3.35)
fuel.add_components({'Li7': 0.0787474673879085,
                  'Be9': 0.0225566879138321,
                  'F19': 0.454003012179284,
                  'Th232': 0.435579130482336,
                  'U233': 0.00911370203663893},
                 percent_type='wo')
fuel.depletable = True
fuel.volume = 48710000.0

moder = openmc.Material(name='graphite', temperature=900.0)
moder.set_density('g/cm3', density=1.84)
moder.add_nuclide('C0', 1.000, percent_type='wo')
moder.add_s_alpha_beta('c_Graphite')

hast = openmc.Material(name='hastelloyN', temperature=900.0)
hast.set_density('g/cm3', density=8.671)
hast.add_components({'Al27': 0.003,
                  'Ni': 0.677,
                  'W': 0.250,
                  'Cr': 0.070},
                 percent_type='wo')

mat = openmc.Materials(materials=[fuel, moder, hast])
mat.export_to_xml()

In [ ]:
colormap = {moder: 'purple',
            hast: 'blue',
            fuel: 'yellow'}

In [ ]:
 def _bound_zone_cells(cells_tuples, levels):
    """Helper function that moves Zone IA and Zone IIA cells to the
    appropriate height."""
    cell_list = []
    n_levels = len(cells_tuples)
    for i, cells in enumerate(cells_tuples):
        if i == 0:
            lower_bound = None
            upper_bound = levels[i]
        elif i == n_levels - 1:
            lower_bound = levels[i-1]
            upper_bound = None
        else:
            lower_bound = levels[i-1]
            upper_bound = levels[i]
        for j, cell in enumerate(cells):
            if lower_bound is None:
                cell.region = cell.region & -upper_bound
            elif upper_bound is None:
                cell.region = cell.region & +lower_bound
            else:
                cell.region = cell.region & +lower_bound & -upper_bound
            cell_list.append(cell)
    return cell_list

def shared_elem_geometry():
    """Surfaces and regions shared by Zone IA and Zone IIA elements. 
    Specs found in Robertson, 1971 Fig 3.4 (p. 17) and Fig 3.5 (p.18)"""

    elem_bound = openmc.rectangular_prism(5.08*2, 5.08*2) # Pin outer boundary
    eb_minx, eb_maxx, eb_miny, eb_maxy = list(elem_bound.get_surfaces().values())

    gr_sq_neg = openmc.rectangular_prism(4.953*2, 4.953*2, corner_radius=0.46) # Graphite square
    (gr_minx, gr_maxx, gr_miny, gr_maxy, 
    gr_cyl_lb, gr_cyl_minx, gr_cyl_miny, 
    gr_cyl_ul, gr_cyl_maxy, gr_cyl_br, 
    gr_cyl_maxx, gr_cyl_ru) = list(gr_sq_neg.get_surfaces().values())
    
    # slabs that line up with rounded edges
    slab_u = -gr_maxy & +gr_cyl_maxy & -gr_cyl_maxx & +gr_cyl_minx
    slab_l = +gr_minx & -gr_cyl_minx & -gr_cyl_maxy & +gr_cyl_miny
    slab_b = +gr_miny & -gr_cyl_miny & -gr_cyl_maxx & +gr_cyl_minx
    slab_r = -gr_maxx & +gr_cyl_maxx & -gr_cyl_maxy & +gr_cyl_miny
    slabs = (slab_u, slab_l, slab_b, slab_r)
    
    # the rounded edges themselves
    quarter_ul = -gr_cyl_ul & -gr_cyl_minx & +gr_cyl_maxy
    quarter_br = -gr_cyl_br & +gr_cyl_maxx & -gr_cyl_miny
    quarter_lb = -gr_cyl_lb & -gr_cyl_minx & -gr_cyl_miny
    quarter_ru = -gr_cyl_ru & +gr_cyl_maxx & +gr_cyl_maxy
    quarters = (quarter_ul, quarter_br, quarter_lb, quarter_ru)
    
    # remaining square
    gr_sq = +gr_cyl_minx & -gr_cyl_maxx & +gr_cyl_miny & -gr_cyl_maxy

    # params for main pin section for both I-A and II-A
    r_d = 0.66802
    l1 = 4.28498
    l2 = 4.53898
    l3 = 5.62102
    ul = openmc.ZCylinder(-l1, l2, r_d, name='corner_ul')
    br = openmc.ZCylinder(l1, -l2, r_d, name='corner_br')
    lb = openmc.ZCylinder(-l2, -l1, r_d, name='corner_lb')
    ru = openmc.ZCylinder(l2, l1, r_d, name='corner_ru')
    ul_t = openmc.ZCylinder(-l1, -l3, r_d, name='corner_ul_tip')
    br_t = openmc.ZCylinder(l1, l3, r_d, name='corner_br_tip')
    ru_t = openmc.ZCylinder(-l3, l1, r_d, name='corner_ru_tip')
    lb_t = openmc.ZCylinder(l3, -l1, r_d, name='corner_lb_tip')
    
    gr_ul = -ul & -eb_maxy & +gr_maxy# | 
    gr_ul_fill = -ul & +gr_cyl_ul & -gr_maxy & -gr_cyl_minx & +gr_cyl_maxy
    gr_br = -br & +eb_miny & -gr_miny# | 
    gr_br_fill = -br & +gr_cyl_br & +gr_miny & +gr_cyl_maxx & -gr_cyl_miny 
    gr_lb = -lb & +eb_minx & -gr_minx# | 
    gr_lb_fill = -lb & +gr_cyl_lb & +gr_minx & -gr_cyl_minx & -gr_cyl_miny 
    gr_ru = -ru & -eb_maxx & +gr_maxx# | 
    gr_ru_fill = -ru & +gr_cyl_ru & -gr_maxx & +gr_cyl_maxx & +gr_cyl_maxy

    gr_ul_t = -ul_t & +eb_miny & -gr_miny
    gr_br_t = -br_t & -eb_maxy & +gr_maxy
    gr_ru_t = -ru_t & +eb_minx & -gr_minx
    gr_lb_t = -lb_t & -eb_maxx & +gr_maxx 
 
    gr_corners = (gr_ul, gr_br, gr_lb, gr_ru, gr_ul_fill, gr_br_fill, gr_lb_fill, gr_ru_fill)
    gr_t = (gr_ul_t, gr_br_t, gr_ru_t, gr_lb_t)
    
    iec_1 = +gr_cyl_miny & -gr_cyl_maxy & +gr_maxx & -eb_maxx & +lb_t & +ru & +gr_cyl_ru
    iec_2 = (+gr_cyl_ru & +gr_cyl_maxx & +gr_cyl_maxy) & +ru & +br_t & -eb_maxx & -eb_maxy
    iec_3 = +gr_cyl_minx & -gr_cyl_maxx & +gr_maxy & -eb_maxy & +br_t & +ul & +gr_cyl_ul
    iec_4 = (+gr_cyl_ul & -gr_cyl_minx & +gr_cyl_maxy) & +ul & +ru_t & +eb_minx & -eb_maxy
    iec_5 = +gr_cyl_miny & -gr_cyl_maxy & -gr_minx & +eb_minx & +ru_t & +lb & +gr_cyl_lb
    iec_6 = (+gr_cyl_lb & -gr_cyl_minx & -gr_cyl_miny) & +lb & +ul_t & +eb_minx & +eb_miny
    iec_7 = +gr_cyl_minx & -gr_cyl_maxx & -gr_miny & +eb_miny & +ul_t & +br & +gr_cyl_br
    iec_8 = (+gr_cyl_br & +gr_cyl_maxx & -gr_cyl_miny) & +br & +lb_t & -eb_maxx & +eb_miny
    
    inter_elem_channel = (iec_1, iec_2, iec_3, iec_4, iec_5, iec_6, iec_7, iec_8)
    #inter_elem_channel = (~gr_sq_neg & elem_bound &
    #                      +ul & +br & +lb & +ru &
    #                      +ul_t & +br_t & +ru_t & +lb_t)
    
    
    gr_round_4 = openmc.ZCylinder(r=2.2225, name='gr_round_4')

    
    return elem_bound, gr_sq, slabs, quarters, gr_corners, gr_t, inter_elem_channel, gr_round_4
                        

In [ ]:
def zoneIA(elem_bound, gr_sq, slabs, quarters, gr_corners, gr_t, inter_elem_channel, gr_round_4):
    """Zone IA element. Specs found in Robertson, 1971 Fig 3.4 (p. 17)"""
    elem_levels = [22.86, 419.10, 438.15, 445.135]
    level_bounds = []
    for level in elem_levels:
        level_bounds.append(openmc.ZPlane(z0=level))
    s1 = openmc.ZCylinder(r=4.953, name='ia_gr_round_1')
    s2 = openmc.ZCylinder(r=1.71069, name='ia_fuel_hole')
    
    h = 12.66
    theta = np.arctan(4.953 / h)
    r2 = (1 / np.cos(theta))**2 - 1
    s3 = openmc.ZCone(z0=h + elem_levels[2], r2=r2, name='cone_i')

    c1 = openmc.Cell(fill=fuel, region=(-s2), name='ia_fuel_inner_1')
    c2 = openmc.Cell(fill=moder, region=(+s2 & -s1), name='ia_moderator_1')
    c3 = openmc.Cell(fill=fuel, region=(+s1 & elem_bound), name='ia_fuel_outer_1')
    ia1 = (c1, c2, c3)
                         
    # I-A  main (lower 2)
    s2 = s2.clone()
    c4 = c1.clone(clone_materials=False)
    c4.name = 'ia_fuel_inner_main'
    c5 = openmc.Cell(fill=moder, region=(+s2 & gr_sq), name='ia_moderator_main')
    #c6 = openmc.Cell(fill=fuel, region=(inter_elem_channel), name='ia_fuel_outer_main')

    c5_slab_u = openmc.Cell(fill=moder, region=slabs[0], name='ia_moderator_main_slab_u')
    c5_slab_l = openmc.Cell(fill=moder, region=slabs[1], name='ia_moderator_main_slab_l')
    c5_slab_b = openmc.Cell(fill=moder, region=slabs[2], name='ia_moderator_main_slab_b')
    c5_slab_r = openmc.Cell(fill=moder, region=slabs[3], name='ia_moderator_main_slab_r')
    
    c5_quarter_u = openmc.Cell(fill=moder, region=quarters[0], name='ia_moderator_main_quarter_u')
    c5_quarter_l = openmc.Cell(fill=moder, region=quarters[1], name='ia_moderator_main_quarter_l')
    c5_quarter_b = openmc.Cell(fill=moder, region=quarters[2], name='ia_moderator_main_quarter_b')
    c5_quarter_r = openmc.Cell(fill=moder, region=quarters[3], name='ia_moderator_main_quarter_r')
    
    c5_ul = openmc.Cell(fill=moder, region=gr_corners[0], name='ia_moderator_main_ul')
    c5_br = openmc.Cell(fill=moder, region=gr_corners[1], name='ia_moderator_main_br')
    c5_ru = openmc.Cell(fill=moder, region=gr_corners[2], name='ia_moderator_main_ru')
    c5_lb = openmc.Cell(fill=moder, region=gr_corners[3], name='ia_moderator_main_lb')
    
    c5_ulf = openmc.Cell(fill=moder, region=gr_corners[4], name='ia_moderator_main_ul_fill')
    c5_brf = openmc.Cell(fill=moder, region=gr_corners[5], name='ia_moderator_main_br_fill')
    c5_ruf = openmc.Cell(fill=moder, region=gr_corners[6], name='ia_moderator_main_ru_fill')
    c5_lbf = openmc.Cell(fill=moder, region=gr_corners[7], name='ia_moderator_main_lb_fill')
    
    c5_ul_t = openmc.Cell(fill=moder, region=gr_t[0], name='ia_moderator_main_ul_t')
    c5_br_t = openmc.Cell(fill=moder, region=gr_t[1], name='ia_moderator_main_br_t')
    c5_ru_t = openmc.Cell(fill=moder, region=gr_t[2], name='ia_moderator_main_ru_t')
    c5_lb_t = openmc.Cell(fill=moder, region=gr_t[3], name='ia_moderator_main_lb_t')
 
    c61 = openmc.Cell(fill=fuel, region=(inter_elem_channel[0]), name='ia_fuel_outer_main_1')
    c62 = openmc.Cell(fill=fuel, region=(inter_elem_channel[1]), name='ia_fuel_outer_main_2')
    c63 = openmc.Cell(fill=fuel, region=(inter_elem_channel[2]), name='ia_fuel_outer_main_3')
    c64 = openmc.Cell(fill=fuel, region=(inter_elem_channel[3]), name='ia_fuel_outer_main_4')
    c65 = openmc.Cell(fill=fuel, region=(inter_elem_channel[4]), name='ia_fuel_outer_main_5')
    c66 = openmc.Cell(fill=fuel, region=(inter_elem_channel[5]), name='ia_fuel_outer_main_6')
    c67 = openmc.Cell(fill=fuel, region=(inter_elem_channel[6]), name='ia_fuel_outer_main_7')
    c68 = openmc.Cell(fill=fuel, region=(inter_elem_channel[7]), name='ia_fuel_outer_main_8')

    iam = (c4, c5,# c6, 
           c5_slab_u, c5_slab_l, c5_slab_b, c5_slab_r,
           c5_quarter_u, c5_quarter_l, c5_quarter_b, c5_quarter_r,
           c5_ul, c5_br, c5_ru, c5_lb, 
           c5_ulf, c5_brf, c5_ruf, c5_lbf, 
           c5_ul_t, c5_br_t, c5_ru_t, c5_lb_t,
          c61, c62, c63, c64, c65, c66, c67, c68)
                         
    # I-A 2 (upper 1)
    c7 = c1.clone(clone_materials=False)
    c7.name = 'ia_fuel_inner_2'
    c8 = c2.clone(clone_materials=False)
    c8.name = 'ia_moderator_2'
    c9 = c3.clone(clone_materials=False)
    c9.name = 'ia_fuel_outer_2'
    ia2 = (c7, c8, c9)

    # I-A 3 (upper 2)'
    s2 = s2.clone()
    s3 = s3.clone()
    elem_bound = elem_bound.clone()
    c10 = c1.clone(clone_materials=False)
    c10.name = 'ia_fuel_inner_3'
    c11 = openmc.Cell(fill=moder, region=(+s2 & -s3), name='ia_moderator_3')
    c12 = openmc.Cell(fill=fuel, region=(+s3 & elem_bound), name='ia_fuel_outer_3')
    ia3 = (c10, c11, c12)                             
                         
    # I-A 4 (upper 3)
    s2 = s2.clone()
    elem_bound = elem_bound.clone()
    c13 = openmc.Cell(fill=hast, region=(-s2), name='ia_hast')
    c14 = openmc.Cell(fill=moder, region=(+s2 & -gr_round_4), name='ia_moderator_4')
    c15 = openmc.Cell(fill=fuel, region=(+gr_round_4 & elem_bound), name='ia_fuel_outer_4')
    ia4 = (c13, c14, c15)
    
    elem_cells = [ia1, iam, ia2, ia3, ia4]
    # universe_id=1
    ia = openmc.Universe(name='zone_ia')
    ia.add_cells(_bound_zone_cells(elem_cells, level_bounds))
                 
    return ia

def zoneIIA(elem_bound, gr_sq, slabs, quarters, gr_corners, gr_t, inter_elem_channel, gr_round_4):
    """Zone IIA element. Specs found in Robertson, 1971 Fig 3.5 (p. 18)"""
    elem_levels = [434.34, 436.88, 439.42, 441.96]
    level_bounds = []
    for level in elem_levels:
        level_bounds.append(openmc.ZPlane(z0=level))
    s1 = openmc.ZCylinder(r=3.302, name='iia_fuel_hole_main') # Hole with fuel salt - Fig 3.5, Roberton 1971 (3.27787 - p.47)
    s2 = openmc.ZCylinder(r=0.635, name='iia_fuel_hole_2')
    s3 = openmc.ZCylinder(r=3.65125, name='iia_gr_round_3')
    h = 6.5
    theta = np.arctan(3.65125 / h)
    r2 = (1 / np.cos(theta))**2 - 1
    s4 = openmc.ZCone(z0=h + elem_levels[2], r2=r2, name='cone_ii')

    # II-A main (lower 1)
    c1 = openmc.Cell(fill=fuel, region=(-s1), name='iia_fuel_inner_main')
    c2 = openmc.Cell(fill=moder, region=(+s1 & gr_sq), name='iia_moderator_main')
    
    c2_slab_u = openmc.Cell(fill=moder, region=slabs[0], name='iia_moderator_main_slab_u')
    c2_slab_l = openmc.Cell(fill=moder, region=slabs[1], name='iia_moderator_main_slab_l')
    c2_slab_b = openmc.Cell(fill=moder, region=slabs[2], name='iia_moderator_main_slab_b')
    c2_slab_r = openmc.Cell(fill=moder, region=slabs[3], name='iia_moderator_main_slab_r')
    
    c2_quarter_u = openmc.Cell(fill=moder, region=quarters[0], name='iia_moderator_main_quarter_u')
    c2_quarter_l = openmc.Cell(fill=moder, region=quarters[1], name='iia_moderator_main_quarter_l')
    c2_quarter_b = openmc.Cell(fill=moder, region=quarters[2], name='iia_moderator_main_quarter_b')
    c2_quarter_r = openmc.Cell(fill=moder, region=quarters[3], name='iia_moderator_main_quarter_r')
    
    c2_ul = openmc.Cell(fill=moder, region=gr_corners[0], name='iia_moderator_main_ul')
    c2_br = openmc.Cell(fill=moder, region=gr_corners[1], name='iia_moderator_main_br')
    c2_ru = openmc.Cell(fill=moder, region=gr_corners[2], name='iia_moderator_main_ru')
    c2_lb = openmc.Cell(fill=moder, region=gr_corners[3], name='iia_moderator_main_lb')
    
    c2_ulf = openmc.Cell(fill=moder, region=gr_corners[4], name='iia_moderator_main_ul_fill')
    c2_brf = openmc.Cell(fill=moder, region=gr_corners[5], name='iia_moderator_main_br_fill')
    c2_ruf = openmc.Cell(fill=moder, region=gr_corners[6], name='iia_moderator_main_ru_fill')
    c2_lbf = openmc.Cell(fill=moder, region=gr_corners[7], name='iia_moderator_main_lb_fill')
    
    # repeat section tips
    c2_ul_t = openmc.Cell(fill=moder, region=gr_t[0], name='iia_moderator_main_ul_t')
    c2_br_t = openmc.Cell(fill=moder, region=gr_t[1], name='iia_moderator_main_br_t')
    c2_ru_t = openmc.Cell(fill=moder, region=gr_t[2], name='iia_moderator_main_ru_t')
    c2_lb_t = openmc.Cell(fill=moder, region=gr_t[3], name='iia_moderator_main_lb_t')
    
    #c3 = openmc.Cell(fill=fuel, region=(inter_elem_channel), name='iia_fuel_outer_main')
    # interelement fuel channel
    c31 = openmc.Cell(fill=fuel, region=(inter_elem_channel[0]), name='iia_fuel_outer_main_1')
    c32 = openmc.Cell(fill=fuel, region=(inter_elem_channel[1]), name='iia_fuel_outer_main_2')
    c33 = openmc.Cell(fill=fuel, region=(inter_elem_channel[2]), name='iia_fuel_outer_main_3')
    c34 = openmc.Cell(fill=fuel, region=(inter_elem_channel[3]), name='iia_fuel_outer_main_4')
    c35 = openmc.Cell(fill=fuel, region=(inter_elem_channel[4]), name='iia_fuel_outer_main_5')
    c36 = openmc.Cell(fill=fuel, region=(inter_elem_channel[5]), name='iia_fuel_outer_main_3')
    c37 = openmc.Cell(fill=fuel, region=(inter_elem_channel[6]), name='iia_fuel_outer_main_7')
    c38 = openmc.Cell(fill=fuel, region=(inter_elem_channel[7]), name='iia_fuel_outer_main_8')

    iiam = (c1, c2, #c3,
            c2_slab_u, c2_slab_l, c2_slab_b, c2_slab_r,
            c2_quarter_u, c2_quarter_l, c2_quarter_b, c2_quarter_r,
            c2_ul, c2_br, c2_ru, c2_lb, 
            c2_ulf, c2_brf, c2_ruf, c2_lbf, 
            c2_ul_t, c2_br_t, c2_ru_t, c2_lb_t,
            c31, c32, c33, c34, c35, c36, c37, c38)

    # II-A 2 (upper 1)
    #gr_sq_neg = gr_sq_neg.clone()
    gr_sq = gr_sq.clone()
    c4 = openmc.Cell(fill=fuel, region=(-s2), name='iia_fuel_inner_2')
    c5 = openmc.Cell(fill=moder, region=(+s2 & gr_sq), name='iia_moderator_2')
    c5_slab_u = c2_slab_u.clone(clone_materials=False)
    c5_slab_u.name = 'iia_moderator_2_slab_u'
    c5_slab_l = c2_slab_l.clone(clone_materials=False)
    c5_slab_l.name = 'iia_moderator_2_slab_l'
    c5_slab_b = c2_slab_b.clone(clone_materials=False)
    c5_slab_b.name = 'iia_moderator_2_slab_b'
    c5_slab_r = c2_slab_r.clone(clone_materials=False)
    c5_slab_r.name = 'iia_moderator_2_slab_r'
    
    c5_quarter_u = c2_quarter_u.clone(clone_materials=False)
    c5_quarter_u.name = 'iia_moderator_2_quarter_u'
    c5_quarter_l = c2_quarter_l.clone(clone_materials=False)
    c5_quarter_l.name = 'iia_moderator_2_quarter_l'
    c5_quarter_b = c2_quarter_b.clone(clone_materials=False)
    c5_quarter_b.name = 'iia_moderator_2_quarter_b'
    c5_quarter_r = c2_quarter_r.clone(clone_materials=False)
    c5_quarter_r.name = 'iia_moderator_2_quarter_r'
    
    c5_ul = c2_ul.clone(clone_materials=False)
    c5_ul.name = 'iia_moderator_2_ul'
    c5_br = c2_br.clone(clone_materials=False)
    c5_br.name = 'iia_moderator_2_br'
    c5_ru = c2_ru.clone(clone_materials=False)
    c5_ru.name = 'iia_moderator_2_ru'
    c5_lb = c2_lb.clone(clone_materials=False)
    c5_lb.name = 'iia_moderator_2_lb'
    c5_ulf = c2_ulf.clone(clone_materials=False)
    c5_ulf.name = 'iia_moderator_2_ulf'
    c5_brf = c2_brf.clone(clone_materials=False)
    c5_brf.name = 'iia_moderator_2_brf'
    c5_ruf = c2_ruf.clone(clone_materials=False)
    c5_ruf.name = 'iia_moderator_2_ruf'
    c5_lbf = c2_lbf.clone(clone_materials=False)
    c5_lbf.name = 'iia_moderator_2_lbf'
    c5_ul_t = c2_ul_t.clone(clone_materials=False)
    c5_ul_t.name = 'iia_moderator_2_ul_t'
    c5_br_t = c2_br_t.clone(clone_materials=False)
    c5_br_t.name = 'iia_moderator_2_br_t'
    c5_ru_t = c2_ru_t.clone(clone_materials=False)
    c5_ru_t.name = 'iia_moderator_2_ru_t'
    c5_lb_t = c2_lb_t.clone(clone_materials=False)
    c5_lb_t.name = 'iia_moderator_2_lb_t'   
    
    c61 = c31.clone(clone_materials=False)
    c61.name = 'iia_fuel_outer_2_1'
    c62 = c32.clone(clone_materials=False)
    c62.name = 'iia_fuel_outer_2_2'
    c63 = c33.clone(clone_materials=False)
    c63.name = 'iia_fuel_outer_2_3'
    c64 = c34.clone(clone_materials=False)
    c64.name = 'iia_fuel_outer_2_4'
    c65 = c35.clone(clone_materials=False)
    c65.name = 'iia_fuel_outer_2_5'
    c66 = c36.clone(clone_materials=False)
    c66.name = 'iia_fuel_outer_2_6'
    c67 = c37.clone(clone_materials=False)
    c67.name = 'iia_fuel_outer_2_7'
    c68 = c38.clone(clone_materials=False)
    c68.name = 'iia_fuel_outer_2_8'
    iia2 = (c4, c5, #c6,
            c5_slab_u, c5_slab_l, c5_slab_b, c5_slab_r,
            c5_quarter_u, c5_quarter_l, c5_quarter_b, c5_quarter_r,
            c5_ul, c5_br, c5_ru, c5_lb,
            c5_ulf, c5_brf, c5_ruf, c5_lbf, 
            c5_ul_t, c5_br_t, c5_ru_t, c5_lb_t,
            c61, c62, c63, c64, c65, c66, c67, c68)

    # II-A 3 (upper 2)
    s2 = s2.clone()
    elem_bound = elem_bound.clone()
    c7 = c4.clone(clone_materials=False)
    c7.name = 'iia_fuel_inner_3'
    c8 = openmc.Cell(fill=moder, region=(+s2 & -s3), name='iia_moderator_3')
    c9 = openmc.Cell(fill=fuel, region=(+s3 & elem_bound), name='iia_fuel_outer_3')
    iia3 = (c7, c8, c9)

    # II-A 4 (upper 3)  
    elem_bound = elem_bound.clone()
    c10 = openmc.Cell(fill=moder, region=(-s4), name='iia_moderator_4')
    c11 = openmc.Cell(fill=fuel, region=(+s4 & elem_bound), name='iia_fuel_outer_4')
    iia4 = (c10, c11)

    # II-A 5 (upper 4)
    elem_bound = elem_bound.clone()
    c12 = openmc.Cell(fill=moder, region=(-gr_round_4), name='iia_moderator_5')
    c13 = openmc.Cell(fill=fuel, region=(+gr_round_4 & elem_bound), name='iia_fuel_outer_5')
    iia5 = (c12, c13)
    
    elem_cells = [iiam, iia2, iia3, iia4, iia5]
    # universe_id=2
    iia = openmc.Universe(name='zone_iia')
    iia.add_cells(_bound_zone_cells(elem_cells, level_bounds))
    return iia

def void_cell(elem_bound):
    elem_bound = elem_bound.clone()
    c1 = openmc.Cell(region=elem_bound, name='lattice_void')
    #universe_id=5
    v = openmc.Universe(name='lattice_void')
    v.add_cell(c1)
    return v

def graphite_triangles(elem_bound):
    s1 = openmc.Plane(1.0, 1.0, 0.0, 0.0)
    s2 = openmc.Plane(-1.0, 1.0, 0.0, 0.0)
    s3 = openmc.Plane(1.0, -1.0, 0.0, 0.0)
    s4 = openmc.Plane(-1.0, -1.0, 0.0, 0.0)
    
    surfs = [(s4, 'bottom_left'),
             (s1, 'top_right'),
             (s2, 'top_left'),
             (s3, 'bottom_right')]
    univs = []
    for i, (s, name) in enumerate(surfs):
        elem_bound = elem_bound.clone()
        c1 = openmc.Cell(fill=moder, region=(-s & elem_bound))
        elem_bound = elem_bound.clone()
        c2 = openmc.Cell(fill=fuel, region=(+s & elem_bound))
    
        # universe_id = 6+i
        gr_tri = openmc.Universe(name=f'{name}_triangle')
        gr_tri.add_cells([c1, c2])
        univs.append(gr_tri)
    
    return univs

In [ ]:
elem_bound = openmc.rectangular_prism(5.08*2, 5.08*2) # Pin outer boundary
eb_minx, eb_maxx, eb_miny, eb_maxy = list(elem_bound.get_surfaces().values())

gr_sq_neg = openmc.rectangular_prism(4.953*2, 4.953*2, corner_radius=0.46) # Graphite square
(gr_minx, gr_maxx, gr_miny, gr_maxy, 
gr_cyl_lb, gr_cyl_minx, gr_cyl_miny, 
gr_cyl_ul, gr_cyl_maxy, gr_cyl_br, 
gr_cyl_maxx, gr_cyl_ru) = list(gr_sq_neg.get_surfaces().values())

# slabs that line up with rounded edges
slab_u = -gr_maxy & +gr_cyl_maxy & -gr_cyl_maxx & +gr_cyl_minx
slab_l = +gr_minx & -gr_cyl_minx & -gr_cyl_maxy & +gr_cyl_miny
slab_b = +gr_miny & -gr_cyl_miny & -gr_cyl_maxx & +gr_cyl_minx
slab_r = -gr_maxx & +gr_cyl_maxx & -gr_cyl_maxy & +gr_cyl_miny
slabs = (slab_u, slab_l, slab_b, slab_r)

# the rounded edges themselves
quarter_ul = -gr_cyl_ul & -gr_cyl_minx & +gr_cyl_maxy
quarter_br = -gr_cyl_br & +gr_cyl_maxx & -gr_cyl_miny
quarter_lb = -gr_cyl_lb & -gr_cyl_minx & -gr_cyl_miny
quarter_ru = -gr_cyl_ru & +gr_cyl_maxx & +gr_cyl_maxy
quarters = (quarter_ul, quarter_br, quarter_lb, quarter_ru)

# remaining square
gr_sq = +gr_cyl_minx & -gr_cyl_maxx & +gr_cyl_miny & -gr_cyl_maxy

# params for main pin section for both I-A and II-A
r_d = 0.66802
l1 = 4.28498
l2 = 4.53898
l3 = 5.62102
ul = openmc.ZCylinder(-l1, l2, r_d, name='corner_ul')
br = openmc.ZCylinder(l1, -l2, r_d, name='corner_br')
lb = openmc.ZCylinder(-l2, -l1, r_d, name='corner_lb')
ru = openmc.ZCylinder(l2, l1, r_d, name='corner_ru')
ul_t = openmc.ZCylinder(-l1, -l3, r_d, name='corner_ul_tip')
br_t = openmc.ZCylinder(l1, l3, r_d, name='corner_br_tip')
ru_t = openmc.ZCylinder(-l3, l1, r_d, name='corner_ru_tip')
lb_t = openmc.ZCylinder(l3, -l1, r_d, name='corner_lb_tip')

gr_ul = -ul & -eb_maxy & +gr_maxy# | 
gr_ul_fill = -ul & +gr_cyl_ul & -gr_maxy & -gr_cyl_minx & +gr_cyl_maxy
gr_br = -br & +eb_miny & -gr_miny# | 
gr_br_fill = -br & +gr_cyl_br & +gr_miny & +gr_cyl_maxx & -gr_cyl_miny 
gr_lb = -lb & +eb_minx & -gr_minx# | 
gr_lb_fill = -lb & +gr_cyl_lb & +gr_minx & -gr_cyl_minx & -gr_cyl_miny 
gr_ru = -ru & -eb_maxx & +gr_maxx# | 
gr_ru_fill = -ru & +gr_cyl_ru & -gr_maxx & +gr_cyl_maxx & +gr_cyl_maxy

gr_ul_t = -ul_t & +eb_miny & -gr_miny
gr_br_t = -br_t & -eb_maxy & +gr_maxy
gr_ru_t = -ru_t & +eb_minx & -gr_minx
gr_lb_t = -lb_t & -eb_maxx & +gr_maxx 

gr_corners = (gr_ul, gr_br, gr_lb, gr_ru, gr_ul_fill, gr_br_fill, gr_lb_fill, gr_ru_fill)
gr_t = (gr_ul_t, gr_br_t, gr_ru_t, gr_lb_t)

iec_1 = +gr_cyl_miny & -gr_cyl_maxy & +gr_maxx & -eb_maxx & +lb_t & +ru & +gr_cyl_ru
iec_2 = (+gr_cyl_ru & +gr_cyl_maxx & +gr_cyl_maxy) & +ru & +br_t & -eb_maxx & -eb_maxy
iec_3 = +gr_cyl_minx & -gr_cyl_maxx & +gr_maxy & -eb_maxy & +br_t & +ul & +gr_cyl_ul
iec_4 = (+gr_cyl_ul & -gr_cyl_minx & +gr_cyl_maxy) & +ul & +ru_t & +eb_minx & -eb_maxy
iec_5 = +gr_cyl_miny & -gr_cyl_maxy & -gr_minx & +eb_minx & +ru_t & +lb & +gr_cyl_lb
iec_6 = (+gr_cyl_lb & -gr_cyl_minx & -gr_cyl_miny) & +lb & +ul_t & +eb_minx & +eb_miny
iec_7 = +gr_cyl_minx & -gr_cyl_maxx & -gr_miny & +eb_miny & +ul_t & +br & +gr_cyl_br
iec_8 = (+gr_cyl_br & +gr_cyl_maxx & -gr_cyl_miny) & +br & +lb_t & -eb_maxx & +eb_miny

inter_elem_channel = (iec_1, iec_2, iec_3, iec_4, iec_5, iec_6, iec_7, iec_8)
#inter_elem_channel = (~gr_sq_neg & elem_bound &
#                      +ul & +br & +lb & +ru &
#                      +ul_t & +br_t & +ru_t & +lb_t)


gr_round_4 = openmc.ZCylinder(r=2.2225, name='gr_round_4')

"""Zone IA element. Specs found in Robertson, 1971 Fig 3.4 (p. 17)"""
elem_levels = [22.86, 419.10, 438.15, 445.135]
level_bounds = []
for level in elem_levels:
    level_bounds.append(openmc.ZPlane(z0=level))
s1 = openmc.ZCylinder(r=4.953, name='ia_gr_round_1')
s2 = openmc.ZCylinder(r=1.71069, name='ia_fuel_hole')

h = 12.66
theta = np.arctan(4.953 / h)
r2 = (1 / np.cos(theta))**2 - 1
s3 = openmc.ZCone(z0=h + elem_levels[2], r2=r2, name='cone_i')

c1 = openmc.Cell(fill=fuel, region=(-s2), name='ia_fuel_inner_1')
c2 = openmc.Cell(fill=moder, region=(+s2 & -s1), name='ia_moderator_1')
c3 = openmc.Cell(fill=fuel, region=(+s1 & elem_bound), name='ia_fuel_outer_1')
ia1 = (c1, c2, c3)

# I-A  main (lower 2)
s2 = s2.clone()
c4 = c1.clone(clone_materials=False)
c4.name = 'ia_fuel_inner_main'
c5 = openmc.Cell(fill=moder, region=(+s2 & gr_sq), name='ia_moderator_main')
#c6 = openmc.Cell(fill=fuel, region=(inter_elem_channel), name='ia_fuel_outer_main')

c5_slab_u = openmc.Cell(fill=moder, region=slabs[0], name='ia_moderator_main_slab_u')
c5_slab_l = openmc.Cell(fill=moder, region=slabs[1], name='ia_moderator_main_slab_l')
c5_slab_b = openmc.Cell(fill=moder, region=slabs[2], name='ia_moderator_main_slab_b')
c5_slab_r = openmc.Cell(fill=moder, region=slabs[3], name='ia_moderator_main_slab_r')

c5_quarter_u = openmc.Cell(fill=moder, region=quarters[0], name='ia_moderator_main_quarter_u')
c5_quarter_l = openmc.Cell(fill=moder, region=quarters[1], name='ia_moderator_main_quarter_l')
c5_quarter_b = openmc.Cell(fill=moder, region=quarters[2], name='ia_moderator_main_quarter_b')
c5_quarter_r = openmc.Cell(fill=moder, region=quarters[3], name='ia_moderator_main_quarter_r')

c5_ul = openmc.Cell(fill=moder, region=gr_corners[0], name='ia_moderator_main_ul')
c5_br = openmc.Cell(fill=moder, region=gr_corners[1], name='ia_moderator_main_br')
c5_ru = openmc.Cell(fill=moder, region=gr_corners[2], name='ia_moderator_main_ru')
c5_lb = openmc.Cell(fill=moder, region=gr_corners[3], name='ia_moderator_main_lb')

c5_ulf = openmc.Cell(fill=moder, region=gr_corners[4], name='ia_moderator_main_ul_fill')
c5_brf = openmc.Cell(fill=moder, region=gr_corners[5], name='ia_moderator_main_br_fill')
c5_ruf = openmc.Cell(fill=moder, region=gr_corners[6], name='ia_moderator_main_ru_fill')
c5_lbf = openmc.Cell(fill=moder, region=gr_corners[7], name='ia_moderator_main_lb_fill')

c5_ul_t = openmc.Cell(fill=moder, region=gr_t[0], name='ia_moderator_main_ul_t')
c5_br_t = openmc.Cell(fill=moder, region=gr_t[1], name='ia_moderator_main_br_t')
c5_ru_t = openmc.Cell(fill=moder, region=gr_t[2], name='ia_moderator_main_ru_t')
c5_lb_t = openmc.Cell(fill=moder, region=gr_t[3], name='ia_moderator_main_lb_t')

c61 = openmc.Cell(fill=fuel, region=(inter_elem_channel[0]), name='ia_fuel_outer_main_1')
c62 = openmc.Cell(fill=fuel, region=(inter_elem_channel[1]), name='ia_fuel_outer_main_2')
c63 = openmc.Cell(fill=fuel, region=(inter_elem_channel[2]), name='ia_fuel_outer_main_3')
c64 = openmc.Cell(fill=fuel, region=(inter_elem_channel[3]), name='ia_fuel_outer_main_4')
c65 = openmc.Cell(fill=fuel, region=(inter_elem_channel[4]), name='ia_fuel_outer_main_5')
c66 = openmc.Cell(fill=fuel, region=(inter_elem_channel[5]), name='ia_fuel_outer_main_6')
c67 = openmc.Cell(fill=fuel, region=(inter_elem_channel[6]), name='ia_fuel_outer_main_7')
c68 = openmc.Cell(fill=fuel, region=(inter_elem_channel[7]), name='ia_fuel_outer_main_8')

iam = (#c4, c5,# c6, 
       #c5_slab_u, c5_slab_l, c5_slab_b, c5_slab_r,
       #c5_quarter_u, c5_quarter_l, c5_quarter_b, c5_quarter_r,
       c5_ul, c5_br, c5_ru, c5_lb, 
       c5_ulf, c5_brf, c5_ruf, c5_lbf, 
       #c5_ul_t, c5_br_t, c5_ru_t, c5_lb_t,
      c61, c62, c63, c64, c65, c66, c67, c68)

In [ ]:
u = openmc.Universe()
u.add_cells(iam)

In [ ]:
u.plot(width=(20,20),
        basis='xy',
        colors=colormap,
        origin=(0.,0., 446),
        #color_by='material',
        pixels=(2000,2000))

In [ ]:
elem_bound, gr_sq, slabs, quarters, gr_corners, gr_t, inter_elem_channel, gr_round_4 = shared_elem_geometry()
ia = zoneIA(elem_bound, gr_sq, slabs, quarters, gr_corners, gr_t, inter_elem_channel, gr_round_4)
elem_bound, gr_sq, slabs, quarters, gr_corners, gr_t, inter_elem_channel, gr_round_4 = shared_elem_geometry()
iia = zoneIIA(elem_bound, gr_sq, slabs, quarters, gr_corners, gr_t, inter_elem_channel, gr_round_4)
# tres, uno, dos, quatro
bl, ur, ul, br = graphite_triangles(elem_bound)

In [ ]:
ia.plot(width=(20,20),
        basis='xz',
        colors=colormap,
        origin=(0.,0., 446),
        color_by='material',
        pixels=(800,800))

In [ ]:
iia.plot(width=(20,20),
        basis='xy',
        colors=colormap,
        origin=(0.,0.,435),
        color_by='material',
        pixels=(800,800))

In [ ]:
def shared_cr_geometry():
    fuel_hole = openmc.ZCylinder(r=5.08, name='cr_fuel_hole')
    
    elem_bound = openmc.rectangular_prism(7.62*2, 7.62*2) # Pin outer boundary
    eb_minx, eb_maxx, eb_miny, eb_maxy = list(elem_bound.get_surfaces().values())
    
    gr_sq_neg = openmc.rectangular_prism(7.23646*2, 7.23646*2, corner_radius=0.99) # Graphite square
    # params for 
    (gr_minx, gr_maxx, gr_miny, gr_maxy, 
    gr_cyl_lb, gr_cyl_minx, gr_cyl_miny, 
    gr_cyl_ul, gr_cyl_maxy, gr_cyl_br, 
    gr_cyl_maxx, gr_cyl_ru) = list(gr_sq_neg.get_surfaces().values())
    
    # slabs that line up with rounded edges
    slab_u = -gr_maxy & +gr_cyl_maxy & -gr_cyl_maxx & +gr_cyl_minx
    slab_l = +gr_minx & -gr_cyl_minx & -gr_cyl_maxy & +gr_cyl_miny
    slab_b = +gr_miny & -gr_cyl_miny & -gr_cyl_maxx & +gr_cyl_minx
    slab_r = -gr_maxx & +gr_cyl_maxx & -gr_cyl_maxy & +gr_cyl_miny
    slabs = (slab_u, slab_l, slab_b, slab_r)
    
    # the rounded edges themselves
    quarter_ul = -gr_cyl_ul & -gr_cyl_minx & +gr_cyl_maxy
    quarter_br = -gr_cyl_br & +gr_cyl_maxx & -gr_cyl_miny
    quarter_lb = -gr_cyl_lb & -gr_cyl_minx & -gr_cyl_miny
    quarter_ru = -gr_cyl_ru & +gr_cyl_maxx & +gr_cyl_maxy
    quarters = (quarter_ul, quarter_br, quarter_lb, quarter_ru)
    
    # remaining square
    gr_sq = +gr_cyl_minx & -gr_cyl_maxx & +gr_cyl_miny & -gr_cyl_maxy
    
    r_d = 1.16
    e_d = 2 * r_d / np.sqrt(3)
    r_dt = 0.8
    r_c = 0.18
    l1 = 5.8801
    l2 = 6.505
    l3 = 8.03646
    ul = openmc.model.hexagonal_prism(origin=(-l1, l2), edge_length=e_d, orientation='x', corner_radius=r_c)
    (ul_u, ul_b, ul_ur, ul_br, ul_bl, ul_ul,
     ul_cyl_bl_in, ul_cyl_bl_out, 
     ul_cyl_ul_in, ul_cyl_ul_out,
     ul_cyl_br_in, ul_cyl_br_out,
     ul_cyl_ur_in, ul_cyl_ur_out,
     ul_cyl_l_in, ul_cyl_l_out,
     ul_cyl_r_in, ul_cyl_r_out)= list(ul.get_surfaces().values())
    br = openmc.model.hexagonal_prism(origin=(l1, -l2), edge_length=e_d, orientation='x',corner_radius=r_c)
    (br_u, br_b, br_ur, br_br, br_bl, br_ul,
     br_cyl_bl_in, br_cyl_bl_out, 
     br_cyl_ul_in, br_cyl_ul_out,
     br_cyl_br_in, br_cyl_br_out,
     br_cyl_ur_in, br_cyl_ur_out,
     br_cyl_l_in, br_cyl_l_out,
     br_cyl_r_in, br_cyl_r_out)= list(br.get_surfaces().values())
    lb = openmc.model.hexagonal_prism(origin=(-l2, -l1), edge_length=e_d, orientation='y',corner_radius=r_c)
    (lb_r, lb_l, lb_ur, lb_ul, lb_br, lb_bl,
     lb_cyl_lb_in, lb_cyl_lb_out, 
     lb_cyl_lu_in, lb_cyl_lu_out,
     lb_cyl_rb_in, lb_cyl_rb_out,
     lb_cyl_ru_in, lb_cyl_ru_out,
     lb_cyl_b_in, lb_cyl_b_out,
     lb_cyl_u_in, lb_cyl_u_out)= list(lb.get_surfaces().values())
    ru = openmc.model.hexagonal_prism(origin=(l2, l1), edge_length=e_d, orientation='y',corner_radius=r_c)
    (ru_r, ru_l, ru_ur, ru_ul, ru_br, ru_bl,
     ru_cyl_lb_in, ru_cyl_lb_out, 
     ru_cyl_lu_in, ru_cyl_lu_out,
     ru_cyl_rb_in, ru_cyl_rb_out,
     ru_cyl_ru_in, ru_cyl_ru_out,
     ru_cyl_b_in, ru_cyl_b_out,
     ru_cyl_u_in, ru_cyl_u_out)= list(ru.get_surfaces().values())
    ul_t = openmc.ZCylinder(-l1, -l3, r_dt, name='corner_ul_tip')
    br_t = openmc.ZCylinder(l1, l3, r_dt, name='corner_br_tip')
    ru_t = openmc.ZCylinder(-l3, l1, r_dt, name='corner_ru_tip')
    lb_t = openmc.ZCylinder(l3, -l1, r_dt, name='corner_lb_tip')
    
    gr_ul = -ul_ul & -ul_ur & -eb_maxy & +gr_maxy
    gr_ul_fill = -ul_ul & +ul_bl & +ul_cyl_l_out & +gr_cyl_ul & -gr_maxy & -gr_cyl_minx & +gr_cyl_maxy
    gr_br = +br_br & +br_bl & +eb_miny & -gr_miny
    gr_br_fill = +br_br & -br_ur & +br_cyl_r_out & +gr_cyl_br & +gr_miny & +gr_cyl_maxx & -gr_cyl_miny
    gr_lb = +lb_bl & -lb_ul & +eb_minx & -gr_minx
    gr_lb_fill = +lb_bl & +lb_br & +lb_cyl_b_out & +gr_cyl_lb & +gr_minx & -gr_cyl_miny & -gr_cyl_minx
    gr_ru = +ru_br & -ru_ur & -eb_maxx & +gr_maxx
    gr_ru_fill = -ru_ur & -ru_ul & +ru_cyl_u_out & +gr_cyl_ru & -gr_maxx & +gr_cyl_maxy & +gr_cyl_maxx


    gr_ul_t = -ul_t & +eb_miny & -gr_miny
    gr_br_t = -br_t & -eb_maxy & +gr_maxy
    gr_ru_t = -ru_t & +eb_minx & -gr_minx
    gr_lb_t = -lb_t & -eb_maxx & +gr_maxx 
 
    gr_corners = (gr_ul, gr_br, gr_lb, gr_ru, gr_ul_fill, gr_br_fill, gr_lb_fill, gr_ru_fill)
    gr_t = (gr_ul_t, gr_br_t, gr_ru_t, gr_lb_t)
    
    # need to decomplexify even #s
    iec_1 = +gr_cyl_miny & +gr_maxx & -eb_maxx & +lb_t & -ru_br
    iec_2 = +gr_cyl_ru & +gr_cyl_maxx & +gr_cyl_maxy & (+ru_ur | +ru_ul | (+ru_cyl_u_in & -ru_cyl_u_out)) & +br_t & -eb_maxx & -eb_maxy
    iec_3 = -gr_cyl_maxx & +gr_maxy & -eb_maxy & +br_t & +ul_ur
    iec_4 = +gr_cyl_ul & -gr_cyl_minx & +gr_cyl_maxy & (+ul_ul | -ul_bl | (+ul_cyl_l_in & -ul_cyl_l_out)) & +ru_t & +eb_minx & -eb_maxy
    iec_5 = -gr_cyl_maxy & -gr_minx & +eb_minx & +ru_t & +lb_ul
    iec_6 = +gr_cyl_lb & -gr_cyl_minx & -gr_cyl_miny & (-lb_bl | -lb_br | (+lb_cyl_b_in & -lb_cyl_b_out)) & +ul_t & +eb_minx & +eb_miny
    iec_7 = +gr_cyl_minx & -gr_miny & +eb_miny & +ul_t & -br_bl
    iec_8 = +gr_cyl_br & +gr_cyl_maxx & -gr_cyl_miny & (-br_br | +br_ur | (+br_cyl_r_in & -br_cyl_r_out)) & +lb_t & -eb_maxx & +eb_miny


    inter_elem_channel = (iec_1, iec_2, iec_3, iec_4, iec_5, iec_6, iec_7, iec_8)
    #inter_elem_channel = (~gr_sq_neg & elem_bound &
    #                     ~ul & ~br & ~lb & ~ru &
    #                     +ul_t & +br_t & +ru_t & +lb_t)
    

    return fuel_hole, gr_sq, slabs, quarters, gr_corners, gr_t, inter_elem_channel
              

In [ ]:
elem_bound = openmc.rectangular_prism(7.62*2, 7.62*2) # Pin outer boundary
eb_minx, eb_maxx, eb_miny, eb_maxy = list(elem_bound.get_surfaces().values())

gr_sq_neg = openmc.rectangular_prism(7.23646*2, 7.23646*2, corner_radius=0.99) # Graphite
    
(gr_minx, gr_maxx, gr_miny, gr_maxy, 
gr_cyl_lb, gr_cyl_minx, gr_cyl_miny, 
gr_cyl_ul, gr_cyl_maxy, gr_cyl_br, 
gr_cyl_maxx, gr_cyl_ru) = list(gr_sq_neg.get_surfaces().values())

# the rounded edges themselves
quarter_ul = -gr_cyl_ul & -gr_cyl_minx & +gr_cyl_maxy
quarter_br = -gr_cyl_br & +gr_cyl_maxx & -gr_cyl_miny
quarter_lb = -gr_cyl_lb & -gr_cyl_minx & -gr_cyl_miny
quarter_ru = -gr_cyl_ru & +gr_cyl_maxx & +gr_cyl_maxy
quarters = (quarter_ul, quarter_br, quarter_lb, quarter_ru)

# remaining square
gr_sq = +gr_cyl_minx & -gr_cyl_maxx & +gr_cyl_miny & -gr_cyl_maxy

r_d = 1.16
e_d = 2 * r_d / np.sqrt(3)
r_dt = 0.8
r_c = 0.18
l1 = 5.8801
l2 = 6.505
l3 = 8.03646
ul = openmc.model.hexagonal_prism(origin=(-l1, l2), edge_length=e_d, orientation='x', corner_radius=r_c)
(ul_u, ul_b, ul_ur, ul_br, ul_bl, ul_ul,
 ul_cyl_bl_in, ul_cyl_bl_out, 
 ul_cyl_ul_in, ul_cyl_ul_out,
 ul_cyl_br_in, ul_cyl_br_out,
 ul_cyl_ur_in, ul_cyl_ur_out,
 ul_cyl_l_in, ul_cyl_l_out,
 ul_cyl_r_in, ul_cyl_r_out)= list(ul.get_surfaces().values())
br = openmc.model.hexagonal_prism(origin=(l1, -l2), edge_length=e_d, orientation='x',corner_radius=r_c)
(br_u, br_b, br_ur, br_br, br_bl, br_ul,
 br_cyl_bl_in, br_cyl_bl_out, 
 br_cyl_ul_in, br_cyl_ul_out,
 br_cyl_br_in, br_cyl_br_out,
 br_cyl_ur_in, br_cyl_ur_out,
 br_cyl_l_in, br_cyl_l_out,
 br_cyl_r_in, br_cyl_r_out)= list(br.get_surfaces().values())
lb = openmc.model.hexagonal_prism(origin=(-l2, -l1), edge_length=e_d, orientation='y',corner_radius=r_c)
(lb_r, lb_l, lb_ur, lb_ul, lb_br, lb_bl,
 lb_cyl_lb_in, lb_cyl_lb_out, 
 lb_cyl_lu_in, lb_cyl_lu_out,
 lb_cyl_rb_in, lb_cyl_rb_out,
 lb_cyl_ru_in, lb_cyl_ru_out,
 lb_cyl_b_in, lb_cyl_b_out,
 lb_cyl_u_in, lb_cyl_u_out)= list(lb.get_surfaces().values())
ru = openmc.model.hexagonal_prism(origin=(l2, l1), edge_length=e_d, orientation='y',corner_radius=r_c)
(ru_r, ru_l, ru_ur, ru_ul, ru_br, ru_bl,
 ru_cyl_lb_in, ru_cyl_lb_out, 
 ru_cyl_lu_in, ru_cyl_lu_out,
 ru_cyl_rb_in, ru_cyl_rb_out,
 ru_cyl_ru_in, ru_cyl_ru_out,
 ru_cyl_b_in, ru_cyl_b_out,
 ru_cyl_u_in, ru_cyl_u_out)= list(ru.get_surfaces().values())
ul_t = openmc.ZCylinder(-l1, -l3, r_dt, name='corner_ul_tip')
br_t = openmc.ZCylinder(l1, l3, r_dt, name='corner_br_tip')
ru_t = openmc.ZCylinder(-l3, l1, r_dt, name='corner_ru_tip')
lb_t = openmc.ZCylinder(l3, -l1, r_dt, name='corner_lb_tip')

gr_ul = -ul_ul & -ul_ur & -eb_maxy & +gr_maxy
gr_ul_fill = -ul_ul & +ul_bl & +ul_cyl_l_out & +gr_cyl_ul & -gr_maxy & -gr_cyl_minx & +gr_cyl_maxy
gr_br = +br_br & +br_bl & +eb_miny & -gr_miny
gr_br_fill = +br_br & -br_ur & +br_cyl_r_out & +gr_cyl_br & +gr_miny & +gr_cyl_maxx & -gr_cyl_miny
gr_lb = +lb_bl & -lb_ul & +eb_minx & -gr_minx
gr_lb_fill = +lb_bl & +lb_br & +lb_cyl_b_out & +gr_cyl_lb & +gr_minx & -gr_cyl_miny & -gr_cyl_minx
gr_ru = +ru_br & -ru_ur & -eb_maxx & +gr_maxx
gr_ru_fill = -ru_ur & -ru_ul & +ru_cyl_u_out & +gr_cyl_ru & -gr_maxx & +gr_cyl_maxy & +gr_cyl_maxx


gr_ul_t = -ul_t & +eb_miny & -gr_miny
gr_br_t = -br_t & -eb_maxy & +gr_maxy
gr_ru_t = -ru_t & +eb_minx & -gr_minx
gr_lb_t = -lb_t & -eb_maxx & +gr_maxx 

gr_corners = (gr_ul, gr_br, gr_lb, gr_ru, gr_ul_fill, gr_br_fill, gr_lb_fill, gr_ru_fill)
gr_t = (gr_ul_t, gr_br_t, gr_ru_t, gr_lb_t)

# need to decomplexify even #s
iec_1 = +gr_cyl_miny & +gr_maxx & -eb_maxx & +lb_t & -ru_br
iec_2 = +gr_cyl_ru & +gr_cyl_maxx & +gr_cyl_maxy & (+ru_ur | +ru_ul | (+ru_cyl_u_in & -ru_cyl_u_out)) & +br_t & -eb_maxx & -eb_maxy
iec_3 = -gr_cyl_maxx & +gr_maxy & -eb_maxy & +br_t & +ul_ur
iec_4 = +gr_cyl_ul & -gr_cyl_minx & +gr_cyl_maxy & (+ul_ul | -ul_bl | (+ul_cyl_l_in & -ul_cyl_l_out)) & +ru_t & +eb_minx & -eb_maxy
iec_5 = -gr_cyl_maxy & -gr_minx & +eb_minx & +ru_t & +lb_ul
iec_6 = +gr_cyl_lb & -gr_cyl_minx & -gr_cyl_miny & (-lb_bl | -lb_br | (+lb_cyl_b_in & -lb_cyl_b_out)) & +ul_t & +eb_minx & +eb_miny
iec_7 = +gr_cyl_minx & -gr_miny & +eb_miny & +ul_t & -br_bl
iec_8 = +gr_cyl_br & +gr_cyl_maxx & -gr_cyl_miny & (-br_br | +br_ur | (+br_cyl_r_in & -br_cyl_r_out)) & +lb_t & -eb_maxx & +eb_miny

inter_elem_channel = (iec_1, iec_2, iec_3, iec_4, iec_5, iec_6, iec_7, iec_8)
#inter_elem_channel = (~gr_sq_neg & elem_bound &

c3 = openmc.Cell(fill=moder, region=(gr_sq), name='cr_moderator')

c3_quarter_u = openmc.Cell(fill=moder, region=quarters[0], name='cr_moderator_quarter_u')
c3_quarter_l = openmc.Cell(fill=moder, region=quarters[1], name='cr_moderator_quarter_l')
c3_quarter_b = openmc.Cell(fill=moder, region=quarters[2], name='cr_moderator_quarter_b')
c3_quarter_r = openmc.Cell(fill=moder, region=quarters[3], name='cr_moderator_quarter_r')

c3_ul = openmc.Cell(fill=moder, region=gr_corners[0], name='cr_moderator_ul')
c3_br = openmc.Cell(fill=moder, region=gr_corners[1], name='cr_moderator_br')
c3_ru = openmc.Cell(fill=moder, region=gr_corners[2], name='cr_moderator_ru')
c3_lb = openmc.Cell(fill=moder, region=gr_corners[3], name='cr_moderator_lb')

c3_ulf = openmc.Cell(fill=moder, region=gr_corners[4], name='cr_moderator_ul_fill')
c3_brf = openmc.Cell(fill=moder, region=gr_corners[5], name='cr_moderator_br_fill')
c3_ruf = openmc.Cell(fill=moder, region=gr_corners[6], name='cr_moderator_ru_fill')
c3_lbf = openmc.Cell(fill=moder, region=gr_corners[7], name='cr_moderator_lb_fill')

c3_ul_t = openmc.Cell(fill=moder, region=gr_t[0], name='cr_moderator_ul_t')
c3_br_t = openmc.Cell(fill=moder, region=gr_t[1], name='cr_moderator_br_t')
c3_ru_t = openmc.Cell(fill=moder, region=gr_t[2], name='cr_moderator_ru_t')
c3_lb_t = openmc.Cell(fill=moder, region=gr_t[3], name='cr_moderator_lb_t')

#c4 = openmc.Cell(fill=fuel, region=inter_elem_channel, name='cr_fuel_outer')
c41 = openmc.Cell(fill=fuel, region=(inter_elem_channel[0]), name='cr_fuel_outer_1')
c42 = openmc.Cell(fill=fuel, region=(inter_elem_channel[1]), name='cr_fuel_outer_2')
c43 = openmc.Cell(fill=fuel, region=(inter_elem_channel[2]), name='cr_fuel_outer_3')
c44 = openmc.Cell(fill=fuel, region=(inter_elem_channel[3]), name='cr_fuel_outer_4')
c45 = openmc.Cell(fill=fuel, region=(inter_elem_channel[4]), name='cr_fuel_outer_5')
c46 = openmc.Cell(fill=fuel, region=(inter_elem_channel[5]), name='cr_fuel_outer_6')
c47 = openmc.Cell(fill=fuel, region=(inter_elem_channel[6]), name='cr_fuel_outer_7')
c48 = openmc.Cell(fill=fuel, region=(inter_elem_channel[7]), name='cr_fuel_outer_8')


#universe_id=3
cr = openmc.Universe(name='control_rod')
cr.add_cells([#c3, #c4,
              #c3_quarter_u, c3_quarter_l, c3_quarter_b, c3_quarter_r,
              c3_ul, c3_br, c3_ru, c3_lb, 
              #c3_ulf, c3_brf, c3_ruf, c3_lbf, 
              c3_ul_t, c3_br_t, c3_ru_t, c3_lb_t,
              c41, c42, c43, c44, c45, c46, c47, c48])

In [ ]:
cr.plot(width=(20,20),
        basis='xy',
        colors=colormap,
        origin=(0.,0.,440),
        #color_by='material',
        pixels=(2000,2000))

In [ ]:
def control_rod(fuel_hole, gr_sq, slabs, quarters, gr_corners, gr_t, inter_elem_channel):
    s1 = openmc.ZCylinder(r=4.7625, name='control_rod')

    c1 = openmc.Cell(fill=moder, region=-s1, name='control_rod')
    c2 = openmc.Cell(fill=fuel, region=(+s1 & -fuel_hole), name='cr_fuel_inner')
    
    c3 = openmc.Cell(fill=moder, region=(+fuel_hole & gr_sq), name='cr_moderator')
    
    c3_slab_u = openmc.Cell(fill=moder, region=slabs[0], name='cr_moderator_slab_u')
    c3_slab_l = openmc.Cell(fill=moder, region=slabs[1], name='cr_moderator_slab_l')
    c3_slab_b = openmc.Cell(fill=moder, region=slabs[2], name='cr_moderator_slab_b')
    c3_slab_r = openmc.Cell(fill=moder, region=slabs[3], name='cr_moderator_slab_r')
    
    c3_quarter_u = openmc.Cell(fill=moder, region=quarters[0], name='cr_moderator_quarter_u')
    c3_quarter_l = openmc.Cell(fill=moder, region=quarters[1], name='cr_moderator_quarter_l')
    c3_quarter_b = openmc.Cell(fill=moder, region=quarters[2], name='cr_moderator_quarter_b')
    c3_quarter_r = openmc.Cell(fill=moder, region=quarters[3], name='cr_moderator_quarter_r')
    
    c3_ul = openmc.Cell(fill=moder, region=gr_corners[0], name='cr_moderator_ul')
    c3_br = openmc.Cell(fill=moder, region=gr_corners[1], name='cr_moderator_br')
    c3_ru = openmc.Cell(fill=moder, region=gr_corners[2], name='cr_moderator_ru')
    c3_lb = openmc.Cell(fill=moder, region=gr_corners[3], name='cr_moderator_lb')
    
    c3_ulf = openmc.Cell(fill=moder, region=gr_corners[4], name='cr_moderator_ul_fill')
    c3_brf = openmc.Cell(fill=moder, region=gr_corners[5], name='cr_moderator_br_fill')
    c3_ruf = openmc.Cell(fill=moder, region=gr_corners[6], name='cr_moderator_ru_fill')
    c3_lbf = openmc.Cell(fill=moder, region=gr_corners[7], name='cr_moderator_lb_fill')
    
    c3_ul_t = openmc.Cell(fill=moder, region=gr_t[0], name='cr_moderator_ul_t')
    c3_br_t = openmc.Cell(fill=moder, region=gr_t[1], name='cr_moderator_br_t')
    c3_ru_t = openmc.Cell(fill=moder, region=gr_t[2], name='cr_moderator_ru_t')
    c3_lb_t = openmc.Cell(fill=moder, region=gr_t[3], name='cr_moderator_lb_t')
    
    #c4 = openmc.Cell(fill=fuel, region=inter_elem_channel, name='cr_fuel_outer')
    c41 = openmc.Cell(fill=fuel, region=(inter_elem_channel[0]), name='cr_fuel_outer_1')
    c42 = openmc.Cell(fill=fuel, region=(inter_elem_channel[1]), name='cr_fuel_outer_2')
    c43 = openmc.Cell(fill=fuel, region=(inter_elem_channel[2]), name='cr_fuel_outer_3')
    c44 = openmc.Cell(fill=fuel, region=(inter_elem_channel[3]), name='cr_fuel_outer_4')
    c45 = openmc.Cell(fill=fuel, region=(inter_elem_channel[4]), name='cr_fuel_outer_5')
    c46 = openmc.Cell(fill=fuel, region=(inter_elem_channel[5]), name='cr_fuel_outer_6')
    c47 = openmc.Cell(fill=fuel, region=(inter_elem_channel[6]), name='cr_fuel_outer_7')
    c48 = openmc.Cell(fill=fuel, region=(inter_elem_channel[7]), name='cr_fuel_outer_8')


    #universe_id=3
    cr = openmc.Universe(name='control_rod')
    cr.add_cells([c1, c2, c3, #c4,
                  c3_slab_u, c3_slab_l, c3_slab_b, c3_slab_r,
                  c3_quarter_u, c3_quarter_l, c3_quarter_b, c3_quarter_r,
                  c3_ul, c3_br, c3_ru, c3_lb, 
                  c3_ulf, c3_brf, c3_ruf, c3_lbf, 
                  c3_ul_t, c3_br_t, c3_ru_t, c3_lb_t,
                  c41, c42, c43, c44, c45, c46, c47, c48])
    
    return cr
    
def control_rod_channel(fuel_hole, gr_sq, slabs, quarters, gr_corners, gr_t, inter_elem_channel):
    c1 = openmc.Cell(fill=fuel, region=(-fuel_hole), name='crc_fuel_inner')
    c2 = openmc.Cell(fill=moder, region=(+fuel_hole & gr_sq), name='crc_moderator')
    
    c2_slab_u = openmc.Cell(fill=moder, region=slabs[0], name='crc_moderator_slab_u')
    c2_slab_l = openmc.Cell(fill=moder, region=slabs[1], name='crc_moderator_slab_l')
    c2_slab_b = openmc.Cell(fill=moder, region=slabs[2], name='crc_moderator_slab_b')
    c2_slab_r = openmc.Cell(fill=moder, region=slabs[3], name='crc_moderator_slab_r')
    
    c2_quarter_u = openmc.Cell(fill=moder, region=quarters[0], name='crc_moderator_quarter_u')
    c2_quarter_l = openmc.Cell(fill=moder, region=quarters[1], name='crc_moderator_quarter_l')
    c2_quarter_b = openmc.Cell(fill=moder, region=quarters[2], name='crc_moderator_quarter_b')
    c2_quarter_r = openmc.Cell(fill=moder, region=quarters[3], name='crc_moderator_quarter_r')
    
    c2_ul = openmc.Cell(fill=moder, region=gr_corners[0], name='crc_moderator_ul')
    c2_br = openmc.Cell(fill=moder, region=gr_corners[1], name='crc_moderator_br')
    c2_ru = openmc.Cell(fill=moder, region=gr_corners[2], name='crc_moderator_ru')
    c2_lb = openmc.Cell(fill=moder, region=gr_corners[3], name='crc_moderator_lb')
    
    c2_ulf = openmc.Cell(fill=moder, region=gr_corners[4], name='crc_moderator_ul_fill')
    c2_brf = openmc.Cell(fill=moder, region=gr_corners[5], name='crc_moderator_br_fill')
    c2_ruf = openmc.Cell(fill=moder, region=gr_corners[6], name='crc_moderator_ru_fill')
    c2_lbf = openmc.Cell(fill=moder, region=gr_corners[7], name='crc_moderator_lb_fill')
    
    c2_ul_t = openmc.Cell(fill=moder, region=gr_t[0], name='crc_moderator_ul_t')
    c2_br_t = openmc.Cell(fill=moder, region=gr_t[1], name='crc_moderator_br_t')
    c2_ru_t = openmc.Cell(fill=moder, region=gr_t[2], name='crc_moderator_ru_t')
    c2_lb_t = openmc.Cell(fill=moder, region=gr_t[3], name='crc_moderator_lb_t')
    
    #c3 = openmc.Cell(fill=fuel, region=inter_elem_channel, name='crc_fuel_outer')
    c31 = openmc.Cell(fill=fuel, region=(inter_elem_channel[0]), name='crc_fuel_outer_1')
    c32 = openmc.Cell(fill=fuel, region=(inter_elem_channel[1]), name='crc_fuel_outer_2')
    c33 = openmc.Cell(fill=fuel, region=(inter_elem_channel[2]), name='crc_fuel_outer_3')
    c34 = openmc.Cell(fill=fuel, region=(inter_elem_channel[3]), name='crc_fuel_outer_4')
    c35 = openmc.Cell(fill=fuel, region=(inter_elem_channel[4]), name='crc_fuel_outer_5')
    c36 = openmc.Cell(fill=fuel, region=(inter_elem_channel[5]), name='crc_fuel_outer_3')
    c37 = openmc.Cell(fill=fuel, region=(inter_elem_channel[6]), name='crc_fuel_outer_7')
    c38 = openmc.Cell(fill=fuel, region=(inter_elem_channel[7]), name='crc_fuel_outer_8')

    


    # universe_id=4
    crc = openmc.Universe(name='control_rod_channel')
    crc.add_cells([c1, c2, #c3, 
                   c2_slab_u, c2_slab_l, c2_slab_b, c2_slab_r,
                   c2_quarter_u, c2_quarter_l, c2_quarter_b, c2_quarter_r,
                   c2_ul, c2_br, c2_ru, c2_lb, 
                   c2_ulf, c2_brf, c2_ruf, c2_lbf, 
                   c2_ul_t, c2_br_t, c2_ru_t, c2_lb_t,
                   c31, c32, c33, c34, c35, c36, c37, c38])
    
    return crc

In [ ]:
fuel_hole, gr_sq, slabs, quarters, gr_corners, gr_t, inter_elem_channel = shared_cr_geometry()          
cr = control_rod(fuel_hole, gr_sq, slabs, quarters, gr_corners, gr_t, inter_elem_channel)
fuel_hole, gr_sq, slabs, quarters, gr_corners, gr_t, inter_elem_channel = shared_cr_geometry()          
crc = control_rod_channel(fuel_hole, gr_sq, slabs, quarters, gr_corners, gr_t, inter_elem_channel)

In [ ]:
cr.cells

In [ ]:
openmc.Universe(cells=[cr.cells[6724]]).plot(width=(20,20),
        basis='xy',
        colors=colormap,
        origin=(0.,0.,440),
        color_by='material',
        pixels=(1000,1000))

In [ ]:
cr.plot(width=(20,20),
        basis='xy',
        colors=colormap,
        origin=(0.,0.,440),
        #color_by='material',
        pixels=(2000,2000))

In [ ]:
crc.plot(width=(20,20),
        basis='xy',
        colors=colormap,
        origin=(0.,0.,440),
        color_by='material',
        pixels=(400,400))

In [ ]:
cr.cells

In [ ]:
c1 = cr.cells[129]
c2 = cr.cells[130]
c3 = cr.cells[131]
c4 = cr.cells[132]
c5 = cr.cells[133]
c6 = cr.cells[134]
c7 = cr.cells[135]
c8 = cr.cells[136]
ttt = openmc.Universe(cells=[])
ttt = openmc.Universe(cells=[c1, c2, c3, c4, c5, c6, c7, c8])
ttt.plot(width=(20,20),
        basis='xy',
        colors=colormap,
        origin=(0.,0.,440),
        color_by='material',
        pixels=(800,800))

In [ ]:
c5 = cr.cells[250]
c6 = cr.cells[251]
c7 = cr.cells[252]
c8 = cr.cells[253]
ttt = openmc.Universe(cells=[c5, c6, c7, c8])
ttt.plot(width=(20,20),
        basis='xy',
        colors=colormap,
        origin=(0.,0.,440),
        color_by='material',
        pixels=(800,800))

In [ ]:
c5

## Figuring out how to decomplexify CR geometry (WIP)

In [ ]:
r_d = 1.16
r_dt = 0.8
e_d = 2 * r_d / np.sqrt(3)
r_c = 0.18
l1 = 5.8801
l2 = 6.505
l3 = 8.03646
ul = openmc.model.hexagonal_prism(origin=(-l1, l2), edge_length=e_d, orientation='x', corner_radius=r_c)
br = openmc.model.hexagonal_prism(origin=(l1, -l2), edge_length=e_d, orientation='x',corner_radius=r_c)
lb = openmc.model.hexagonal_prism(origin=(-l2, -l1), edge_length=e_d, orientation='y',corner_radius=r_c)
ru = openmc.model.hexagonal_prism(origin=(l2, l1), edge_length=e_d, orientation='y',corner_radius=r_c)
gr_sq_neg = openmc.rectangular_prism(7.23646*2, 7.23646*2, corner_radius=0.99) # Graphite square
elem_bound = openmc.rectangular_prism(7.62*2, 7.62*2) # Pin outer boundary

ul_t = openmc.ZCylinder(-l1, -l3, r_dt, name='corner_ul_tip')
br_t = openmc.ZCylinder(l1, l3, r_dt, name='corner_br_tip')
ru_t = openmc.ZCylinder(-l3, l1, r_dt, name='corner_ru_tip')
lb_t = openmc.ZCylinder(l3, -l1, r_dt, name='corner_lb_tip')
c1 = openmc.Cell(fill=moder, region=ul)
c2 = openmc.Cell(fill=moder, region=br)
c3 = openmc.Cell(fill=moder, region=lb)
c4 = openmc.Cell(fill=moder, region=ru)
c5 = openmc.Cell(fill=hast, region=gr_sq_neg)
c6 = openmc.Cell(fill=fuel, region=-ul_t)
c7 = openmc.Cell(fill=fuel, region=-br_t)
c8 = openmc.Cell(fill=fuel, region=-ru_t)
c9 = openmc.Cell(fill=fuel, region=-lb_t)
c10 = openmc.Cell(fill=moder, region=elem_bound)

tu = openmc.Universe(cells=[c1, c2, c3, c4, c5, c6, c7, c8, c9, c10])

In [ ]:
ul = openmc.model.hexagonal_prism(origin=(-l1, l2), edge_length=e_d, orientation='x', corner_radius=r_c)
# for x oriented
(ul_u, ul_b, ul_ur, ul_br, ul_bl, ul_ul,
 ul_cyl_bl_in, ul_cyl_bl_out, 
 ul_cyl_ul_in, ul_cyl_ul_out,
 ul_cyl_br_in, ul_cyl_br_out,
 ul_cyl_ur_in, ul_cyl_ur_out,
 ul_cyl_l_in, ul_cyl_l_out,
 ul_cyl_r_in, ul_yl_r_out)= list(ul.get_surfaces().values())

In [ ]:
c3

In [ ]:
(lb_r, lb_l, lb_ur, lb_ul, lb_br, lb_bl,
 lb_cyl_lb_in, lb_cyl_lb_out, 
 lb_cyl_lu_in, lb_cyl_lu_out,
 lb_cyl_rb_in, lb_cyl_rb_out,
 lb_cyl_ru_in, lb_cyl_ru_out,
 lb_cyl_b_in, lb_cyl_b_out,
 lb_cyl_u_in, lb_yl_u_out)= list(lb.get_surfaces().values())

In [ ]:
lb.get_surfaces()

In [ ]:
#t = openmc.Region.from_expression('(-1448 1449 -1450 1451 1452 -1453 ~((1454 -1460) | (1455 -1461) | (1456 -1462) | (1457 -1463) | (1458 -1464) | (1459 -1465)))', c1.region.get_surfaces())
#t = openmc.Region.from_expression('(-1448 1449 -1450 1451 1452 -1453 ((-1454 | 1460) (-1455 | 1461) (-1456 | 1462) (-1457 | 1463) (-1458 | 1464) (-1459 | 1465)))', c1.region.get_surfaces())
t = openmc.Region.from_expression('(-772 773 -774 -775 776 777 (778 -784))', c3.region.get_surfaces())

#t = openmc.Region.from_expression('((-1454 | -1455 | -1456 | -1457 | -1458 | -1459) | 1460 1461 1462 1463 1464 1465)', c1.region.get_surfaces())
#t = openmc.Region.from_expression('(-1448 1449 -1450 1451 1452 -1453 (-1454 | -1455 | -1456 | -1457 | -1458 | -1459 | 1460 1461 1462 1463 1464 1465))', c1.region.get_surfaces())


In [ ]:
tt = openmc.Cell(fill=moder, region=t)
ttt = openmc.Universe(cells=[tt])
ttt.plot(width=(20,20),
        basis='xy',
        colors=colormap,
        origin=(0.,0.,440),
        color_by='material',
        pixels=(800,800))

In [ ]:
tu.plot(width=(20,20),
        basis='xy',
        colors=colormap,
        origin=(0.,0.,440),
        #color_by='material',
        pixels=(800,800))

In [ ]:
s1 = openmc.model.IsogonalOctagon(center=(0.0,0.0), r1=208.28, r2=222.71, name='base_octader')
s2 = openmc.model.IsogonalOctagon(center=(0.0,0.0), r1=218.44, r2=215.53, name='smaller_octader')
s3 = openmc.model.IsogonalOctagon(center=(0.0,0.0), r1=228.60, r2=193.97, name='smallest_octader')
oct1 = list((-s1).get_surfaces().values())
oct2 = list((-s2).get_surfaces().values())
oct3 = list((-s3).get_surfaces().values())
octas = (oct1, oct2, oct3)
oct1_maxy, oct1_miny, oct1_maxx, oct1_minx, oct1_ur, oct1_br, oct1_bl, oct1_ul = octas[0]
oct2_maxy, oct2_miny, oct2_maxx, oct2_minx, oct2_ur, oct2_br, oct2_bl, oct2_ul = octas[1]
oct3_maxy, oct3_miny, oct3_maxx, oct3_minx, oct3_ur, oct3_br, oct3_bl, oct3_ul = octas[2]

cr_boundary = openmc.model.rectangular_prism(15.24*2, 15.24*2)

cb_minx, cb_maxx, cb_miny, cb_maxy = list(cr_boundary.get_surfaces().values())

#c1 = openmc.Cell(fill=moder, region=+s3 & -s2)
c1_ur = openmc.Cell(fill=moder, region=(-oct3_ur & +oct2_ur & -oct2_maxx & -oct2_maxy))
c1_ul = openmc.Cell(fill=moder, region=(+oct3_ul & -oct2_ul & +oct2_minx & -oct2_maxy))
c1_bl = openmc.Cell(fill=moder, region=(+oct3_bl & -oct2_bl & +oct2_minx & +oct2_miny))
c1_br = openmc.Cell(fill=moder, region=(-oct3_br & +oct2_br & -oct2_maxx & +oct2_miny))
c1 = [c1_ur, c1_ul, c1_bl, c1_br]

#c2 = openmc.Cell(fill=moder, region=-s3)
c2_r = openmc.Cell(fill=moder, region=(+cb_maxx & +cb_miny & -cb_maxy & -oct3_maxx))
c2_ur = openmc.Cell(fill=moder, region=(+cb_maxx & +cb_maxy & -oct3_maxx & -oct3_maxy & +oct3_ur))
c2_u = openmc.Cell(fill=moder, region=(+cb_maxy & +cb_minx & -cb_maxx & -oct3_maxy))
c2_ul = openmc.Cell(fill=moder, region=(-cb_minx & +cb_maxy & +oct3_minx & -oct3_maxy & -oct3_ul))
c2_l = openmc.Cell(fill=moder, region=(-cb_minx & +cb_miny & -cb_maxy & +oct3_minx))
c2_bl = openmc.Cell(fill=moder, region=(-cb_minx & -cb_miny & +oct3_minx & +oct3_miny & -oct3_bl))
c2_b = openmc.Cell(fill=moder, region=(-cb_miny & +cb_minx & -cb_maxx & +oct3_miny))
c2_br = openmc.Cell(fill=moder, region=(+cb_maxx & -cb_miny & -oct3_maxx & +oct3_miny & +oct3_br))


c2 = [c2_r, c2_ur, c2_u, c2_ul, c2_l, c2_bl, c2_b, c2_br]

#c3 = openmc.Cell(fill=moder, region=-s1 & +s2)
c3_ur = openmc.Cell(fill=moder, region=(-oct2_ur & +oct1_ur & -oct1_maxx & -oct1_maxy))
c3_ul = openmc.Cell(fill=moder, region=(+oct2_ul & -oct1_ul & +oct1_minx & -oct1_maxy))
c3_bl = openmc.Cell(fill=moder, region=(+oct2_bl & -oct1_bl & +oct1_minx & +oct1_miny))
c3_br = openmc.Cell(fill=moder, region=(-oct2_br & +oct1_br & -oct1_maxx & +oct1_miny))
c3 = [c3_ur, c3_ul, c3_bl, c3_br]

u = openmc.Universe()
#u.add_cells(c1)
#u.add_cells(c2)
u.add_cells(c3)

u.plot(width=(700,700),
        basis='xy',
        #colors={c1: 'red', c2:'blue', c3:'green'},
        origin=(0.,0.,200),
        #color_by='material',
        pixels=(1000,1000))

In [ ]:
s1 = openmc.model.IsogonalOctagon(center=(0.0,0.0), r1=208.28, r2=222.71, name='base_octader')
s2 = openmc.model.IsogonalOctagon(center=(0.0,0.0), r1=218.44, r2=215.53, name='smaller_octader')
s3 = openmc.model.IsogonalOctagon(center=(0.0,0.0), r1=228.60, r2=193.97, name='smallest_octader')
oct1 = list((-s1).get_surfaces().values())
oct2 = list((-s2).get_surfaces().values())
oct3 = list((-s3).get_surfaces().values())
octas = (oct1, oct2, oct3)
oct1_maxy, oct1_miny, oct1_maxx, oct1_minx, oct1_ur, oct1_br, oct1_bl, oct1_ul = octas[0]
oct2_maxy, oct2_miny, oct2_maxx, oct2_minx, oct2_ur, oct2_br, oct2_bl, oct2_ul = octas[1]
oct3_maxy, oct3_miny, oct3_maxx, oct3_minx, oct3_ur, oct3_br, oct3_bl, oct3_ul = octas[2]

zone_ii_boundary = openmc.ZCylinder(r=256.032, name='iib_boundary')

cr_boundary = openmc.model.rectangular_prism(15.24*2, 15.24*2)
cb_minx, cb_maxx, cb_miny, cb_maxy = list(cr_boundary.get_surfaces().values())


rr = openmc.Cell(fill=moder, region=(+oct3_maxx & +cb_miny & -cb_maxy & -zone_ii_boundary))
uu = openmc.Cell(fill=moder, region=(+oct3_maxy & +cb_minx & -cb_maxx & -zone_ii_boundary))
ll = openmc.Cell(fill=moder, region=(-oct3_minx & +cb_miny & -cb_maxy & -zone_ii_boundary))
bb = openmc.Cell(fill=moder, region=(-oct3_miny & +cb_minx & -cb_maxx & -zone_ii_boundary))

#c1 = openmc.Cell(fill=moder, region=((+s3 & +s2 & +s1) & -zone_ii_boundary))
ur = openmc.Cell(fill=moder, region=((-oct1_ur | +oct1_maxx | +oct1_maxy) &
                                    (-oct2_ur | +oct2_maxx | +oct2_maxy) &
                                    (-oct3_ur | +oct3_maxx | +oct3_maxy) & -zone_ii_boundary
                                    & +cb_maxx & +cb_maxy))
ul = openmc.Cell(fill=moder, region=((+oct1_ul | -oct1_minx | +oct1_maxy) &
                                    (+oct2_ul | -oct2_minx | +oct2_maxy) &
                                    (+oct3_ul | -oct3_minx | +oct3_maxy) & -zone_ii_boundary
                                    & -cb_minx & +cb_maxy))
bl = openmc.Cell(fill=moder, region=((+oct1_bl | -oct1_minx | -oct1_miny) &
                                    (+oct2_bl | -oct2_minx | -oct2_miny) &
                                    (+oct3_bl | -oct3_minx | -oct3_miny) & -zone_ii_boundary
                                    & -cb_minx & -cb_miny))
br = openmc.Cell(fill=moder, region=((-oct1_br | +oct1_maxx | -oct1_miny) &
                                    (-oct2_br | +oct2_maxx | -oct2_miny) &
                                    (-oct3_br | +oct3_maxx | -oct3_miny) & -zone_ii_boundary
                                    & +cb_maxx & -cb_miny))
#c3_ul = openmc.Cell(fill=moder, region=(+oct2_ul & -oct1_ul & +oct1_minx & -oct1_maxy))
#c3_bl = openmc.Cell(fill=moder, region=(+oct2_bl & -oct1_bl & +oct1_minx & +oct1_miny))
#c3_br = openmc.Cell(fill=moder, region=(-oct2_br & +oct1_br & -oct1_maxx & +oct1_miny))
#c1_ur = openmc.Cell(fill=moder, region=(-oct3_ur & +oct1_maxx & +oct1_maxy & -zone_ii_boundary))
#c1 = [c1_ur]
#c2 = openmc.Cell(fill=moder, region=+s1 & -zone_ii_boundary)


#c2 = [c2_r, c2_ur, c2_u, c2_ul, c2_l, c2_bl, c2_b, c2_br]

#c3 = openmc.Cell(fill=moder, region=-s1 & +s2)

u = openmc.Universe()
#u.add_cells(c1)
u.add_cells([ur, ul, bl, br, ll, rr, bb, uu])
#u.add_cells(c3)

u.plot(width=(700,700),
        basis='xy',
        #colors={c1: 'red', c2:'blue', c3:'green'},
        origin=(0.,0.,200),
        #color_by='material',
        pixels=(1000,1000))

In [ ]:
def shared_root_geometry():
    cr_boundary = openmc.model.rectangular_prism(15.24*2, 15.24*2)
    
    core_base = openmc.ZPlane(z0=0.0, name='core_base')
    core_top = openmc.ZPlane(z0=449.58, name='core_top')
    
    s1 = openmc.model.IsogonalOctagon(center=(0.0,0.0), r1=208.28, r2=222.71, name='base_octader')
    s2 = openmc.model.IsogonalOctagon(center=(0.0,0.0), r1=218.44, r2=215.53, name='smaller_octader')
    s3 = openmc.model.IsogonalOctagon(center=(0.0,0.0), r1=228.60, r2=193.97, name='smallest_octader')
    
    oct1 = list((-s1).get_surfaces().values())
    oct2 = list((-s2).get_surfaces().values())
    oct3 = list((-s3).get_surfaces().values())
    zone_i_octas = (oct1, oct2, oct3)
    
    zone_i_boundary = (s1, s2, s3)

    zone_ii_boundary = openmc.ZCylinder(r=256.032, name='iib_boundary')
    annulus_boundary = openmc.ZCylinder(r=261.112, name='annulus_boundary')
    lower_plenum_boundary = openmc.ZPlane(z0=-7.62, name='lower_plenum_boundary')

    zone_bounds = (cr_boundary, zone_i_boundary, zone_i_octas, zone_ii_boundary)
    
    core_bounds = (annulus_boundary, lower_plenum_boundary, core_base, core_top)
    
    radial_reflector_boundary = openmc.ZCylinder(r=338.328, name='radial_reflector_boundary')    
    bottom_reflector_boundary = openmc.ZPlane(z0=-76.2, name='bottom_axial_reflector_boundary')
    top_reflector_boundary = openmc.ZPlane(z0=525.78, name='top_axial_reflector_boundary') 
    
    reflector_bounds = (radial_reflector_boundary,
                        bottom_reflector_boundary,
                        top_reflector_boundary)
    
    radial_vessel_boundary = openmc.ZCylinder(r=343.408, name='radial_vessel_wall', boundary_type='vacuum')
    bottom_vessel_boundary = openmc.ZPlane(z0=-81.28, name='bottom_vessel_wall', boundary_type='vacuum')
    top_vessel_boundary = openmc.ZPlane(z0=530.86, name='top_vessel_wall', boundary_type='vacuum')
    
    vessel_bounds = (radial_vessel_boundary,
                     bottom_vessel_boundary,
                     top_vessel_boundary)
    
    return zone_bounds, core_bounds, reflector_bounds, vessel_bounds

def cr_lattice(cr_boundary, core_base, core_top):
    fuel_hole, gr_sq, slabs, quarters, gr_corners, gr_t, inter_elem_channel = shared_cr_geometry()          
    f = control_rod(fuel_hole, gr_sq, slabs, quarters, gr_corners, gr_t, inter_elem_channel)
    fuel_hole, gr_sq, slabs, quarters, gr_corners, gr_t, inter_elem_channel = shared_cr_geometry()          
    e = control_rod_channel(fuel_hole, gr_sq, slabs, quarters, gr_corners, gr_t, inter_elem_channel)
    
    cr = openmc.RectLattice()
    cr.pitch = np.array([15.24, 15.24])
    N = 2 / 2
    cr.lower_left = -1 * cr.pitch * N
    cr.universes = [[f, e],
                    [e, f]]
    
    c1 = openmc.Cell(fill=cr, region=(+core_base & -core_top & cr_boundary), name='cr_lattice')
    
    return c1

def main_lattice(zone_i_octas, cr_boundary, core_base, core_top):
    elem_bound, gr_sq, slabs, quarters, gr_corners, gr_t, inter_elem_channel, gr_round_4 = shared_elem_geometry()
    l = zoneIA(elem_bound, gr_sq, slabs, quarters, gr_corners, gr_t, inter_elem_channel, gr_round_4)
    elem_bound, gr_sq, slabs, quarters, gr_corners, gr_t, inter_elem_channel, gr_round_4 = shared_elem_geometry()
    z = zoneIIA(elem_bound, gr_sq, slabs, quarters, gr_corners, gr_t, inter_elem_channel, gr_round_4)
    v = void_cell(elem_bound)
    # tres, uno, dos, quatro
    t, u, d, q = graphite_triangles(elem_bound) 
    
    main = openmc.RectLattice()
    main.pitch = np.array([10.16, 10.16])
    N = 45 / 2
    main.lower_left = -1 * main.pitch * N
    main.universes = [[v, v, v, v, v, v, v, v, v, v, v, v, v, v, v, v, v, d, z, z, z, z, z, z, z, z, z, u, v, v, v, v, v, v, v, v, v, v, v, v, v, v, v, v, v],
                      [v, v, v, v, v, v, v, v, v, v, v, v, v, d, z, z, z, z, l, l, l, l, l, l, l, l, l, z, z, z, z, u, v, v, v, v, v, v, v, v, v, v, v, v, v],
                      [v, v, v, v, v, v, v, v, v, v, v, d, z, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, z, u, v, v, v, v, v, v, v, v, v, v, v],
                      [v, v, v, v, v, v, v, v, v, v, d, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, u, v, v, v, v, v, v, v, v, v, v],
                      [v, v, v, v, v, v, v, v, v, d, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, u, v, v, v, v, v, v, v, v, v],
                      [v, v, v, v, v, v, v, v, d, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, u, v, v, v, v, v, v, v, v],
                      [v, v, v, v, v, v, v, d, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, u, v, v, v, v, v, v, v],
                      [v, v, v, v, v, v, d, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, u, v, v, v, v, v, v],
                      [v, v, v, v, v, d, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, u, v, v, v, v, v],
                      [v, v, v, v, d, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, u, v, v, v, v],
                      [v, v, v, d, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, u, v, v, v],
                      [v, v, d, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, u, v, v],
                      [v, v, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, v, v],
                      [v, d, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, u, v],
                      [v, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, v],
                      [v, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, v],
                      [v, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, v],
                      [d, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, u],
                      [z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z],
                      [z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z],
                      [z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z],
                      [z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, v, v, v, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z],
                      [z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, v, v, v, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z],
                      [z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, v, v, v, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z],
                      [z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z],
                      [z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z],
                      [z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z],
                      [t, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, q],
                      [v, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, v],
                      [v, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, v],
                      [v, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, v],
                      [v, t, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, q, v],
                      [v, v, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, v, v],
                      [v, v, t, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, q, v, v],
                      [v, v, v, t, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, q, v, v, v],
                      [v, v, v, v, t, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, q, v, v, v, v],
                      [v, v, v, v, v, t, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, q, v, v, v, v, v],
                      [v, v, v, v, v, v, t, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, q, v, v, v, v, v, v],
                      [v, v, v, v, v, v, v, t, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, q, v, v, v, v, v, v, v],
                      [v, v, v, v, v, v, v, v, t, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, q, v, v, v, v, v, v, v, v],
                      [v, v, v, v, v, v, v, v, v, t, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, q, v, v, v, v, v, v, v, v, v],
                      [v, v, v, v, v, v, v, v, v, v, t, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, q, v, v, v, v, v, v, v, v, v, v],
                      [v, v, v, v, v, v, v, v, v, v, v, t, z, z, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, l, z, z, q, v, v, v, v, v, v, v, v, v, v, v],
                      [v, v, v, v, v, v, v, v, v, v, v, v, v, t, z, z, z, z, l, l, l, l, l, l, l, l, l, z, z, z, z, q, v, v, v, v, v, v, v, v, v, v, v, v, v],
                      [v, v, v, v, v, v, v, v, v, v, v, v, v, v, v, v, v, t, z, z, z, z, z, z, z, z, z, q, v, v, v, v, v, v, v, v, v, v, v, v, v, v, v, v, v]]
    
    oct1_maxy, oct1_miny, oct1_maxx, oct1_minx, oct1_ur, oct1_br, oct1_bl, oct1_ul = zone_i_octas[0]
    oct2_maxy, oct2_miny, oct2_maxx, oct2_minx, oct2_ur, oct2_br, oct2_bl, oct2_ul = zone_i_octas[1]
    oct3_maxy, oct3_miny, oct3_maxx, oct3_minx, oct3_ur, oct3_br, oct3_bl, oct3_ul = zone_i_octas[2]
    cb_minx, cb_maxx, cb_miny, cb_maxy = list(cr_boundary.get_surfaces().values())
    
    #c1 = openmc.Cell(fill=moder, region=+s3 & -s2)
    c1_ur = openmc.Cell(fill=main, region=(-oct3_ur & +oct2_ur & -oct2_maxx & -oct2_maxy),
                       name='smaller_octader_ur')
    c1_ul = openmc.Cell(fill=main, region=(+oct3_ul & -oct2_ul & +oct2_minx & -oct2_maxy),
                       name='smaller_octader_ul')
    c1_bl = openmc.Cell(fill=main, region=(+oct3_bl & -oct2_bl & +oct2_minx & +oct2_miny),
                       name='smaller_octader_bl')
    c1_br = openmc.Cell(fill=main, region=(-oct3_br & +oct2_br & -oct2_maxx & +oct2_miny),
                       name='smaller_octader_br')
    c1 = [c1_ur, c1_ul, c1_bl, c1_br]

    #c2 = openmc.Cell(fill=moder, region=-s3)
    c2_r = openmc.Cell(fill=main, region=(+cb_maxx & +cb_miny & -cb_maxy & -oct3_maxx),
                       name='smallest_octader_r')
    c2_ur = openmc.Cell(fill=main, region=(+cb_maxx & +cb_maxy & -oct3_maxx & -oct3_maxy & +oct3_ur),
                       name='smallest_octader_ur')
    c2_u = openmc.Cell(fill=main, region=(+cb_maxy & +cb_minx & -cb_maxx & -oct3_maxy),
                       name='smallest_octader_u')
    c2_ul = openmc.Cell(fill=main, region=(-cb_minx & +cb_maxy & +oct3_minx & -oct3_maxy & -oct3_ul),
                       name='smallest_octader_ul')
    c2_l = openmc.Cell(fill=main, region=(-cb_minx & +cb_miny & -cb_maxy & +oct3_minx),
                       name='smallest_octader_l')
    c2_bl = openmc.Cell(fill=main, region=(-cb_minx & -cb_miny & +oct3_minx & +oct3_miny & -oct3_bl),
                       name='smallest_octader_bl')
    c2_b = openmc.Cell(fill=main, region=(-cb_miny & +cb_minx & -cb_maxx & +oct3_miny),
                       name='smallest_octader_b')
    c2_br = openmc.Cell(fill=main, region=(+cb_maxx & -cb_miny & -oct3_maxx & +oct3_miny & +oct3_br),
                       name='smallest_octader_br')
    c2 = [c2_r, c2_ur, c2_u, c2_ul, c2_l, c2_bl, c2_b, c2_br]

    #c3 = openmc.Cell(fill=moder, region=-s1 & +s2)
    c3_ur = openmc.Cell(fill=main, region=(-oct2_ur & +oct1_ur & -oct1_maxx & -oct1_maxy),
                       name='base_octader_ur')
    c3_ul = openmc.Cell(fill=main, region=(+oct2_ul & -oct1_ul & +oct1_minx & -oct1_maxy),
                       name='base_octader_ul')
    c3_bl = openmc.Cell(fill=main, region=(+oct2_bl & -oct1_bl & +oct1_minx & +oct1_miny),
                       name='base_octader_bl')
    c3_br = openmc.Cell(fill=main, region=(-oct2_br & +oct1_br & -oct1_maxx & +oct1_miny),
                       name='base_octader_br')
    c3 = [c3_ur, c3_ul, c3_bl, c3_br]
    for cells in (c1, c2, c3):
        for cell in cells:
            cell.region = cell.region & +core_base & -core_top 

    #c1 = openmc.Cell(fill=main, region=(+core_base & -core_top & 
    #                                    +zone_i_boundary[2] & -zone_i_boundary[1] & 
    #                                    ~cr_boundary), name='main_lattice')
    #c2 = openmc.Cell(fill=main, region=(+core_base & -core_top & 
    #                                    -zone_i_boundary[2] & 
    #                                    ~cr_boundary), name='main_lattice')
    #c3 = openmc.Cell(fill=main, region=(+core_base & -core_top & 
    #                                    -zone_i_boundary[0] & +zone_i_boundary[1] & +zone_i_boundary[2] &
    #                                    ~cr_boundary), name='main_lattice')
    
    return c1, c2, c3

def zoneIIB(zone_i_octas, zone_i_boundary, cr_boundary, zone_ii_boundary, annulus_boundary, core_base, core_top):
    # Large elements
    large_angular_width = 3.538
    large_half_w = large_angular_width / 2
    large_positions = np.linspace(0, 315, 8)
    r_outer = 256.032
    r_big1 = 229.6
    r_big2 = 223.6
    rb_1 = (r_big1, r_outer)
    rb_2 = (r_big2, r_outer)
    big_radii = [rb_1, rb_2] * 4
    small_radii = (207.28, r_outer)
    
    r_hole = 3.0875
    hole_args = ({'x0': 242.679, 'y0': 0.0, 'r': r_hole},
                 {'x0': 171.60, 'y0': 171.60, 'r': r_hole},
                 {'x0': 0.0, 'y0': 242.679, 'r': r_hole},
                 {'x0': -171.60, 'y0': 171.60, 'r': r_hole},
                 {'x0': -242.679, 'y0': 0.0, 'r': r_hole},
                 {'x0': -171.60, 'y0': -171.60, 'r': r_hole},
                 {'x0': 0.0, 'y0': -242.697, 'r': r_hole},
                 {'x0': 171.60, 'y0': -171.60, 'r': r_hole})
    
    # Small elements
    small_angular_width = 0.96
    adjacent_angular_offset = 0.675 #27/40
    small_elems_per_octant = 25
    
    elem_cells = []
    for i, pos in enumerate(large_positions):
        pos = np.round(pos, 3)
        r1_big, r2_big = big_radii[i]
        t1_big = pos - large_half_w
        t2_big = pos + large_half_w
        s1 = openmc.model.CylinderSector(r1_big, r2_big, t1_big, t2_big)
        s2 = openmc.ZCylinder(**hole_args[i])
        elem_cells.append(openmc.Cell(fill=moder, region=(-s1 & +s2), name=f'iib_large_element_{pos}'))
        elem_cells.append(openmc.Cell(fill=fuel, region=(-s2),
                                      name=f'iib_large_element_fuel_hole_{pos}'))
        t1_small = t2_big + adjacent_angular_offset
        r1_small, r2_small = small_radii
        
        # Inter element fuel channel
        s3 = openmc.model.CylinderSector(r1_small, r2_small, t2_big, t1_small)
        cpos = t2_big + (adjacent_angular_offset / 2)
        cpos = np.round(cpos, 3)
        elem_cells.append(openmc.Cell(fill=fuel, region=-s3,
                                     name=f'inter_element_fuel_channel_{cpos}'))
        
        t4a = t1_big - adjacent_angular_offset
        s4 = openmc.model.CylinderSector(r1_small, r1_big, t4a, t1_small)
        elem_cells.append(openmc.Cell(fill=fuel, region=-s4,
                                     name=f'inter_element_fuel_channel_{pos}'))
        
        for i in range(0, small_elems_per_octant):
            t2_small = t1_small + small_angular_width
            
            # reflector element
            s5 = openmc.model.CylinderSector(r1_small, r2_small, t1_small, t2_small)
            pos = t2_small - (small_angular_width / 2)
            pos = np.round(pos, 3)
            elem_cells.append(openmc.Cell(fill=moder, region=-s5, name=f'iib_small_element_{pos}'))
            t1_small = t2_small + adjacent_angular_offset
            
            # inter-element fuel channel
            s6 = openmc.model.CylinderSector(r1_small, r2_small, t2_small, t1_small)
            cpos = t2_small + (adjacent_angular_offset/2)
            cpos = np.round(cpos, 3)
            elem_cells.append(openmc.Cell(fill=fuel, region=-s6,
                                         name=f'inter_element_fuel_channel_{cpos}'))
                    

    #universe_id=10
    iib = openmc.Universe(name='zone_iib')
    iib.add_cells(elem_cells)
    
    oct1_maxy, oct1_miny, oct1_maxx, oct1_minx, oct1_ur, oct1_br, oct1_bl, oct1_ul = zone_i_octas[0]
    oct2_maxy, oct2_miny, oct2_maxx, oct2_minx, oct2_ur, oct2_br, oct2_bl, oct2_ul = zone_i_octas[1]
    oct3_maxy, oct3_miny, oct3_maxx, oct3_minx, oct3_ur, oct3_br, oct3_bl, oct3_ul = zone_i_octas[2]
    cb_minx, cb_maxx, cb_miny, cb_maxy = list(cr_boundary.get_surfaces().values())
    
    
    c1_r = openmc.Cell(fill=iib, region=(+oct3_maxx & +cb_miny & -cb_maxy & -zone_ii_boundary),
                      name='zone_ii_b_r')
    c1_ur = openmc.Cell(fill=iib, region=((-oct1_ur | +oct1_maxx | +oct1_maxy) &
                                        (-oct2_ur | +oct2_maxx | +oct2_maxy) &
                                        (-oct3_ur | +oct3_maxx | +oct3_maxy) & -zone_ii_boundary
                                        & +cb_maxx & +cb_maxy),
                      name='zone_ii_b_ur')
    c1_u = openmc.Cell(fill=iib, region=(+oct3_maxy & +cb_minx & -cb_maxx & -zone_ii_boundary),
                      name='zone_ii_b_u')
    c1_ul = openmc.Cell(fill=iib, region=((+oct1_ul | -oct1_minx | +oct1_maxy) &
                                        (+oct2_ul | -oct2_minx | +oct2_maxy) &
                                        (+oct3_ul | -oct3_minx | +oct3_maxy) & -zone_ii_boundary
                                        & -cb_minx & +cb_maxy),
                      name='zone_ii_b_ul')
    c1_l = openmc.Cell(fill=iib, region=(-oct3_minx & +cb_miny & -cb_maxy & -zone_ii_boundary),
                      name='zone_ii_b_l')
    c1_bl = openmc.Cell(fill=iib, region=((+oct1_bl | -oct1_minx | -oct1_miny) &
                                        (+oct2_bl | -oct2_minx | -oct2_miny) &
                                        (+oct3_bl | -oct3_minx | -oct3_miny) & -zone_ii_boundary
                                        & -cb_minx & -cb_miny),
                      name='zone_ii_b_bl')
    c1_b = openmc.Cell(fill=iib, region=(-oct3_miny & +cb_minx & -cb_maxx & -zone_ii_boundary),
                      name='zone_ii_b_b')
    c1_br = openmc.Cell(fill=iib, region=((-oct1_br | +oct1_maxx | -oct1_miny) &
                                        (-oct2_br | +oct2_maxx | -oct2_miny) &
                                        (-oct3_br | +oct3_maxx | -oct3_miny) & -zone_ii_boundary
                                        & +cb_maxx & -cb_miny),
                      name='zone_ii_b_br')
    c1 = openmc.Cell(fill=iib, region=(+zone_i_boundary[0] & +zone_i_boundary[1] & +zone_i_boundary[2] & 
                                       -zone_ii_boundary & 
                                       +core_base & 
                                       -core_top), name='zone_iib')
    #c1 = [c1_r, c1_ur, c1_u, c1_ul, c1_l, c1_bl, c1_b, c1_br]
    #for c in c1:
    #    c.region = c.region & +core_base & -core_top

                        
    return [c1]

def annulus(zone_ii_boundary, annulus_boundary, core_base, core_top):
    annulus_reg = +zone_ii_boundary & -annulus_boundary & +core_base & -core_top
    c1 = openmc.Cell(fill=fuel, region=annulus_reg, name='annulus')
    
    return c1

def lower_plenum(core_base, lower_plenum_boundary, annulus_boundary):
    lower_plenum_reg = -core_base & +lower_plenum_boundary & -annulus_boundary 
    c1 = openmc.Cell(fill=fuel, region=lower_plenum_reg, name='lower_plenum')

    return c1

def reflectors(annulus_boundary, 
               radial_reflector_boundary, 
               lower_plenum_boundary,
               bottom_reflector_boundary, 
               core_top, 
               top_reflector_boundary):
    
    radial_reflector_reg = +annulus_boundary & -radial_reflector_boundary & +bottom_reflector_boundary & -top_reflector_boundary
    bottom_reflector_reg = -annulus_boundary & -lower_plenum_boundary & +bottom_reflector_boundary
    top_reflector_reg = -annulus_boundary & +core_top & -top_reflector_boundary

    c1 = openmc.Cell(fill=moder, region=radial_reflector_reg, name='radial_reflector')
    c2 = openmc.Cell(fill=moder, region=bottom_reflector_reg, name='bottom_axial_reflector')
    c3 = openmc.Cell(fill=moder, region=top_reflector_reg, name='top_axial_reflector')
    
    return c1, c2, c3

def vessel(radial_reflector_boundary,
           radial_vessel_boundary,
           bottom_vessel_boundary,
           top_vessel_boundary,
           top_reflector_boundary,
           bottom_reflector_boundary):
    radial_vessel_reg = +radial_reflector_boundary & -radial_vessel_boundary & -top_vessel_boundary & +bottom_vessel_boundary
    bottom_vessel_reg = -radial_reflector_boundary & -bottom_reflector_boundary & +bottom_vessel_boundary
    top_vessel_reg = -radial_reflector_boundary & -top_vessel_boundary & +top_reflector_boundary

    
    c1 = openmc.Cell(fill=hast, region=radial_vessel_reg, name='radial_vessel_wall')
    c2 = openmc.Cell(fill=hast, region=bottom_vessel_reg, name='bottom_vessel_wall')
    c3 = openmc.Cell(fill=hast, region=top_vessel_reg, name='top_vessel_wall')
    
    return c1, c2, c3

In [ ]:
t = -zone_i_boundary[0]

In [ ]:
zone_i_boundary[0].bottom

In [ ]:
maxy, miny, maxx, minx, ur, br, bl, ul = list(t.get_surfaces().values())

In [ ]:
zone_bounds, core_bounds, reflector_bounds, vessel_bounds = shared_root_geometry()

cr_boundary, zone_i_boundary, zone_i_octas, zone_ii_boundary = zone_bounds
annulus_boundary, lower_plenum_boundary, core_base, core_top = core_bounds
radial_reflector_boundary, bottom_reflector_boundary, top_reflector_boundary = reflector_bounds
radial_vessel_boundary, bottom_vessel_boundary, top_vessel_boundary= vessel_bounds

main1, main2, main3 = main_lattice(zone_i_octas, cr_boundary, core_base, core_top)
cr = cr_lattice(cr_boundary, core_base, core_top)
iib = zoneIIB(zone_i_octas, zone_i_boundary, cr_boundary, zone_ii_boundary, annulus_boundary, core_base, core_top)
a = annulus(zone_ii_boundary, annulus_boundary, core_base, core_top)
lp = lower_plenum(core_base, lower_plenum_boundary, annulus_boundary)

rr, rb, rt = reflectors(annulus_boundary, 
               radial_reflector_boundary, 
               lower_plenum_boundary,
               bottom_reflector_boundary, 
               core_top, 
               top_reflector_boundary)

vr, vb, vt = vessel(radial_reflector_boundary,
           radial_vessel_boundary,
           bottom_vessel_boundary,
           top_vessel_boundary,
           top_reflector_boundary,
           bottom_reflector_boundary)

testuniverse = openmc.Universe()
testuniverse.add_cells([cr, lp, a, rr, rb, rt, vr, vb, vt])
testuniverse.add_cells(main1)
testuniverse.add_cells(main2)
testuniverse.add_cells(main3)
testuniverse.add_cells(iib)


In [ ]:
iib.fill.plot(width=(700,700),
        basis='xy',
        colors=colormap,
        origin=(0.,0.,200),
        color_by='material',
        pixels=(1000,1000))

In [ ]:
testuniverse.plot(width=(700,700),
        basis='xy',
        colors=colormap,
        origin=(0.,0.,200),
        color_by='material',
        pixels=(1000,1000))

In [ ]:
#testuniverse.plot(width=(700,650),
#        basis='xz',
#        colors=colormap,
#        origin=(0.,0.,220),
#        color_by='material',
#        pixels=(1000,1000))

In [ ]:
geo = openmc.Geometry()
geo.root_universe = testuniverse
geo.remove_redundant_surfaces()
#geo.export_to_xml()

In [ ]:
geo.export_to_xml()

In [ ]:
geo.get_all_cells()

In [ ]:
plots = openmc.Plots()

plot = openmc.Plot(name='detail-zoneIA-IIA-lower1')
plot.origin=(215, 0, 10.0)
plot.width=(40, 40)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
plots.append(plot)

plot = openmc.Plot(name='detail-zoneIA-main')
plot.origin=(215, 0, 23.0)
plot.width=(40, 40)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
plots.append(plot)

plot = openmc.Plot(name='detail-zoneIIA-upper1')
plot.origin=(215, 0, 435)
plot.width=(40, 40)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
plots.append(plot)

plot = openmc.Plot(name='detail-zoneIA-upper1')
plot.origin=(215, 0, 420)
plot.width=(40, 40)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
plots.append(plot)

plot = openmc.Plot(name='detail-zoneIIA-upper2')
plot.origin=(215, 0, 437)
plot.width=(40, 40)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
plots.append(plot)

plot = openmc.Plot(name='detail-zoneIA-upper2')
plot.origin=(215, 0, 439)
plot.width=(40, 40)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
plots.append(plot)

plot = openmc.Plot(name='detail-zoneIIA-upper3')
plot.origin=(215, 0, 440)
plot.width=(40, 40)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
plots.append(plot)

plot = openmc.Plot(name='detail-zoneIA-upper3')
plot.origin=(215, 0, 448)
plot.width=(40, 40)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
plots.append(plot)

plot = openmc.Plot(name='detail-zoneIIA-upper4')
plot.origin=(215, 0, 442)
plot.width=(40, 40)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
plots.append(plot)

plot = openmc.Plot(name='full-zoneIA-IIA-lower1')
plot.origin=(0.0, 0, 10.0)
plot.width=(600, 600)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
plots.append(plot)

plot = openmc.Plot(name='full-zoneIA-main')
plot.origin=(0, 0, 23.0)
plot.width=(600, 600)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
plots.append(plot)

plot = openmc.Plot(name='full-zoneIIA-upper1')
plot.origin=(0, 0, 435)
plot.width=(600, 600)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
plots.append(plot)

plot = openmc.Plot(name='full-zoneIA-upper1')
plot.origin=(0, 0, 420)
plot.width=(600, 600)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
plots.append(plot)

plot = openmc.Plot(name='full-zoneIIA-upper2')
plot.origin=(0, 0, 437)
plot.width=(600, 600)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
plots.append(plot)

plot = openmc.Plot(name='full-zoneIA-upper2')
plot.origin=(0, 0, 439)
plot.width=(600, 600)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
plots.append(plot)

plot = openmc.Plot(name='full-zoneIIA-upper3')
plot.origin=(0, 0, 440)
plot.width=(600, 600)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
plots.append(plot)

plot = openmc.Plot(name='full-zoneIA-upper3')
plot.origin=(0, 0, 448)
plot.width=(600, 600)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
plots.append(plot)

plot = openmc.Plot(name='full-zoneIIA-upper4')
plot.origin=(0, 0, 442)
plot.width=(600, 600)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
plots.append(plot)

plot = openmc.Plot(name='core-xz-detail-upper')
plot.origin=(215, 0, 440)
plot.width=(100, 100)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xz'
plots.append(plot)

plot = openmc.Plot(name='full-core-xz')
plot.origin=(0, 0, 200)
plot.width=(700, 700)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xz'
plots.append(plot)

plots.export_to_xml()